# 0- Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
try:
    from amplpy import AMPL, tools
except ImportError:
    # Caso ocorra um erro ao importar, instala amplpy e tenta importar novamente
    %pip install amplpy
    from amplpy import AMPL, tools

In [3]:
licenca ="default"

In [4]:

ampl = tools.ampl_notebook(
    modules=["highs", "gcg", "gurobi", "cplex"],  # Módulos a serem instalados
    license_uuid=licenca,  # Licença a ser utilizada
    g=globals()  # Instancia o objeto AMPL e registra os comandos mágicos
)


AMPL Development Version 20240606 (MSVC 19.40.33811.0, 64-bit)
Demo license with maintenance expiring 20260131.
Using license file "c:\Users\canda\Desktop\CANDACE\GITHUB\Resolucao_Problemas\.venv\Lib\site-packages\ampl_module_base\bin\ampl.lic".



# 1- Base

Importando a base pré-processada com o limite de produção diário de:
<ul>
    <li>120.000 peças para o “Tipo de buffer” CX e </li>
    <li>100.000 peças para o “Tipo de buffer” EN.</li>
 </ul>

In [5]:
df_process = pd.read_excel('../../01-Data/ordens_pre_process_revisado.xlsx')

df_process

,Unnamed: 0,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Tipo_de_buffer,Item_pai,Cor,Tamanho,Inner,Rota,Capacidade,pai_cor,chave_loja_buffer_onda,Volume_unit
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,CX,1865,6.0,27,1,1,inf,1865_6.0,26_CX_5,3092300
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,CX,1865,6.0,21,1,1,inf,1865_6.0,26_CX_5,3321500
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,CX,1865,6.0,4,1,1,inf,1865_6.0,26_CX_5,3537200
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,CX,2489,674.0,179,1,3,inf,2489_674.0,116_CX_6,233624
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,CX,2988,451.0,109,1,1,inf,2988_451.0,16_CX_2,775500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,EN,1863,2.0,38,1,15,4.000000e+10,1863_2.0,48_EN_2,3269280
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,EN,1863,2.0,58,1,15,4.000000e+10,1863_2.0,48_EN_2,4037285
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,EN,1863,2.0,16,1,15,4.000000e+10,1863_2.0,48_EN_2,3231360
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,EN,713,275.0,16,1,15,4.000000e+10,713_275.0,48_EN_2,4235707


In [6]:
df_process = df_process.rename(columns={'Unnamed: 0': "Id"})

In [7]:
df_process = df_process.rename(columns={'Capacidade': "Capacidade_rota"})

In [8]:
df_process.columns

Index(['Id', 'Ordem_de_produção', 'Data_da_ordem_de_produção', 'Produto',
       'Comprimento_unit', 'Largura_unit', 'Altura_unit', 'Peso_unit', 'Peças',
       'Caixa_padrão', 'Loja', 'Classe_de_onda', 'Tipo_de_buffer', 'Item_pai',
       'Cor', 'Tamanho', 'Inner', 'Rota', 'Capacidade_rota', 'pai_cor',
       'chave_loja_buffer_onda', 'Volume_unit'],
      dtype='object')

In [9]:
df_process['tempo_espera_dias'] = (df_process['Data_da_ordem_de_produção'].max() - df_process['Data_da_ordem_de_produção']).dt.days.astype('int16')

In [10]:
# Definir a função para calcular a prioridade
def calcular_prioridade(delta_t):
    diferenca = delta_t_maximo - delta_t
    if diferenca <= 5:
        return delta_t_maximo
    elif diferenca <= 10:
        return delta_t_maximo - 10
    else:
        return delta_t_maximo - 20

In [11]:
delta_t_maximo = df_process['tempo_espera_dias'].max()
df_process['prioridade'] = df_process['tempo_espera_dias'].apply(calcular_prioridade)
df_process

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Cor,Tamanho,Inner,Rota,Capacidade_rota,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,prioridade
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,6.0,27,1,1,inf,1865_6.0,26_CX_5,3092300,98,98
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,6.0,21,1,1,inf,1865_6.0,26_CX_5,3321500,98,98
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,6.0,4,1,1,inf,1865_6.0,26_CX_5,3537200,98,98
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,674.0,179,1,3,inf,2489_674.0,116_CX_6,233624,37,78
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,451.0,109,1,1,inf,2988_451.0,16_CX_2,775500,37,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,2.0,38,1,15,4.000000e+10,1863_2.0,48_EN_2,3269280,0,78
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,2.0,58,1,15,4.000000e+10,1863_2.0,48_EN_2,4037285,0,78
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,2.0,16,1,15,4.000000e+10,1863_2.0,48_EN_2,3231360,0,78
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,275.0,16,1,15,4.000000e+10,713_275.0,48_EN_2,4235707,0,78


In [12]:
(df_process['Data_da_ordem_de_produção'].max() - df_process['Data_da_ordem_de_produção']).dt.days.astype('int16').head(300)

0      98
1      98
2      98
3      37
4      37
       ..
295    20
296    20
297    20
298    20
299    20
Name: Data_da_ordem_de_produção, Length: 300, dtype: int16

In [13]:
df_process.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62231 entries, 0 to 62230
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Id                         62231 non-null  int64         
 1   Ordem_de_produção          62231 non-null  int64         
 2   Data_da_ordem_de_produção  62231 non-null  datetime64[ns]
 3   Produto                    62231 non-null  int64         
 4   Comprimento_unit           62231 non-null  int64         
 5   Largura_unit               62231 non-null  int64         
 6   Altura_unit                62231 non-null  int64         
 7   Peso_unit                  62231 non-null  float64       
 8   Peças                      62231 non-null  int64         
 9   Caixa_padrão               62231 non-null  object        
 10  Loja                       62231 non-null  int64         
 11  Classe_de_onda             62231 non-null  int64         
 12  Tipo

In [14]:
df_process

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Cor,Tamanho,Inner,Rota,Capacidade_rota,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,prioridade
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,6.0,27,1,1,inf,1865_6.0,26_CX_5,3092300,98,98
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,6.0,21,1,1,inf,1865_6.0,26_CX_5,3321500,98,98
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,6.0,4,1,1,inf,1865_6.0,26_CX_5,3537200,98,98
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,674.0,179,1,3,inf,2489_674.0,116_CX_6,233624,37,78
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,451.0,109,1,1,inf,2988_451.0,16_CX_2,775500,37,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,2.0,38,1,15,4.000000e+10,1863_2.0,48_EN_2,3269280,0,78
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,2.0,58,1,15,4.000000e+10,1863_2.0,48_EN_2,4037285,0,78
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,2.0,16,1,15,4.000000e+10,1863_2.0,48_EN_2,3231360,0,78
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,275.0,16,1,15,4.000000e+10,713_275.0,48_EN_2,4235707,0,78


## Caixa padrão

In [15]:
df_caixas_tipo = pd.read_excel('../../01-Data/Dados - desafio disciplina RPVMM_edit.xlsx', sheet_name='Tipos de caixa')
df_caixas_tipo

,Tipo,Tipo de buffer,Comprimento,Largura,Altura,Fator de ocupação,Peças max,Peso max (kg)
0,Caixa 1,CX,380,285,285,0.9,170,23
1,Caixa 2,CX,586,436,272,1.2,170,23
2,Caixa 3,CX,728,427,270,0.9,170,23
3,Caixa 1,EN,380,285,285,0.9,512,23
4,Caixa 2,EN,586,436,272,1.2,512,23
5,Caixa 3,EN,728,427,270,0.9,512,23


In [16]:
df_caixas_tipo.columns = df_caixas_tipo.columns.str.replace(' ', '_')

In [17]:
df_caixas_tipo['volume_util_caixa'] = df_caixas_tipo['Comprimento'] * df_caixas_tipo['Largura'] * df_caixas_tipo['Altura'] * df_caixas_tipo['Fator_de_ocupação']

df_caixas_tipo

,Tipo,Tipo_de_buffer,Comprimento,Largura,Altura,Fator_de_ocupação,Peças_max,Peso_max_(kg),volume_util_caixa
0,Caixa 1,CX,380,285,285,0.9,170,23,27778950.0
1,Caixa 2,CX,586,436,272,1.2,170,23,83393894.4
2,Caixa 3,CX,728,427,270,0.9,170,23,75538008.0
3,Caixa 1,EN,380,285,285,0.9,512,23,27778950.0
4,Caixa 2,EN,586,436,272,1.2,512,23,83393894.4
5,Caixa 3,EN,728,427,270,0.9,512,23,75538008.0


In [18]:
df_caixas_tipo.pivot(index='Tipo_de_buffer', columns=['Tipo','volume_util_caixa'], values=['Peças_max', 'volume_util_caixa']).reset_index()

Tipo_de_buffer  Peças_max                        \
Tipo                                Caixa 1    Caixa 2    Caixa 3   
volume_util_caixa                27778950.0 83393894.4 75538008.0   
0                             CX      170.0      170.0      170.0   
1                             EN      512.0      512.0      512.0   

                  volume_util_caixa                          
Tipo                        Caixa 1     Caixa 2     Caixa 3  
volume_util_caixa        27778950.0  83393894.4  75538008.0  
0                        27778950.0  83393894.4  75538008.0  
1                        27778950.0  83393894.4  75538008.0

In [19]:
df_caixas_tipo.pivot(index='Tipo_de_buffer', columns='Tipo', values='volume_util_caixa').add_prefix('volume_util_caixa_').reset_index()

Tipo,Tipo_de_buffer,volume_util_caixa_Caixa 1,volume_util_caixa_Caixa 2,volume_util_caixa_Caixa 3
0,CX,27778950.0,83393894.4,75538008.0
1,EN,27778950.0,83393894.4,75538008.0


In [20]:
df_caixas_tipo.pivot(index='Tipo_de_buffer', columns='Tipo', values='Peças_max').add_prefix('Peças_max_').reset_index()

Tipo,Tipo_de_buffer,Peças_max_Caixa 1,Peças_max_Caixa 2,Peças_max_Caixa 3
0,CX,170,170,170
1,EN,512,512,512


In [21]:
df_caixas_tipo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Tipo               6 non-null      object 
 1   Tipo_de_buffer     6 non-null      object 
 2   Comprimento        6 non-null      int64  
 3   Largura            6 non-null      int64  
 4   Altura             6 non-null      int64  
 5   Fator_de_ocupação  6 non-null      float64
 6   Peças_max          6 non-null      int64  
 7   Peso_max_(kg)      6 non-null      int64  
 8   volume_util_caixa  6 non-null      float64
dtypes: float64(2), int64(5), object(2)
memory usage: 564.0+ bytes


In [22]:
df_caixas_tipo.columns

Index(['Tipo', 'Tipo_de_buffer', 'Comprimento', 'Largura', 'Altura',
       'Fator_de_ocupação', 'Peças_max', 'Peso_max_(kg)', 'volume_util_caixa'],
      dtype='object')

Temos um tipo de caixa padrão para cada tipo de buffer

In [23]:
df_caixas_padrao = df_caixas_tipo.loc[[1, 5]]
df_caixas_padrao

,Tipo,Tipo_de_buffer,Comprimento,Largura,Altura,Fator_de_ocupação,Peças_max,Peso_max_(kg),volume_util_caixa
1,Caixa 2,CX,586,436,272,1.2,170,23,83393894.4
5,Caixa 3,EN,728,427,270,0.9,512,23,75538008.0


Levar a capacidade das caixas para cada ordem de produção:

O **número de peças máximo** varia com o buffer. Já o *volume* é igual para ambos os tipos, mudando apenas para o **tipo de caixa**

In [24]:
df_caixa_volume_max= df_caixas_tipo.pivot(index='Tipo_de_buffer', columns='Tipo', values='volume_util_caixa').add_prefix('volume_util_caixa_').reset_index()

df_caixa_volume_max

Tipo,Tipo_de_buffer,volume_util_caixa_Caixa 1,volume_util_caixa_Caixa 2,volume_util_caixa_Caixa 3
0,CX,27778950.0,83393894.4,75538008.0
1,EN,27778950.0,83393894.4,75538008.0


In [25]:
df_process_capacitado = df_process.merge(df_caixa_volume_max, on='Tipo_de_buffer', how='inner')

df_process_capacitado

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Rota,Capacidade_rota,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa 1,volume_util_caixa_Caixa 2,volume_util_caixa_Caixa 3
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,1,inf,1865_6.0,26_CX_5,3092300,98,98,27778950.0,83393894.4,75538008.0
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,1,inf,1865_6.0,26_CX_5,3321500,98,98,27778950.0,83393894.4,75538008.0
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,1,inf,1865_6.0,26_CX_5,3537200,98,98,27778950.0,83393894.4,75538008.0
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,3,inf,2489_674.0,116_CX_6,233624,37,78,27778950.0,83393894.4,75538008.0
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,1,inf,2988_451.0,16_CX_2,775500,37,78,27778950.0,83393894.4,75538008.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,15,4.000000e+10,1863_2.0,48_EN_2,3269280,0,78,27778950.0,83393894.4,75538008.0
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,15,4.000000e+10,1863_2.0,48_EN_2,4037285,0,78,27778950.0,83393894.4,75538008.0
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,15,4.000000e+10,1863_2.0,48_EN_2,3231360,0,78,27778950.0,83393894.4,75538008.0
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,15,4.000000e+10,713_275.0,48_EN_2,4235707,0,78,27778950.0,83393894.4,75538008.0


In [26]:
df_process_capacitado.columns = df_process_capacitado.columns.str.replace(' ', '_')

In [27]:
import numpy as np
df_process_capacitado['volume_util_padrao'] = np.where(df_process_capacitado['Tipo_de_buffer'] == "EN", df_process_capacitado['volume_util_caixa_Caixa_3'], df_process_capacitado['volume_util_caixa_Caixa_2'])

In [28]:
df_process_capacitado = df_process_capacitado.merge(df_caixas_padrao[['Tipo_de_buffer', 'Peças_max']], on='Tipo_de_buffer', how='inner')
df_process_capacitado

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,1865_6.0,26_CX_5,3092300,98,98,27778950.0,83393894.4,75538008.0,83393894.4,170
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,1865_6.0,26_CX_5,3321500,98,98,27778950.0,83393894.4,75538008.0,83393894.4,170
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,1865_6.0,26_CX_5,3537200,98,98,27778950.0,83393894.4,75538008.0,83393894.4,170
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,2489_674.0,116_CX_6,233624,37,78,27778950.0,83393894.4,75538008.0,83393894.4,170
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,2988_451.0,16_CX_2,775500,37,78,27778950.0,83393894.4,75538008.0,83393894.4,170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,1863_2.0,48_EN_2,3269280,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,1863_2.0,48_EN_2,4037285,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,1863_2.0,48_EN_2,3231360,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,713_275.0,48_EN_2,4235707,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512


In [29]:
df_process_capacitado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62231 entries, 0 to 62230
Data columns (total 29 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Id                         62231 non-null  int64         
 1   Ordem_de_produção          62231 non-null  int64         
 2   Data_da_ordem_de_produção  62231 non-null  datetime64[ns]
 3   Produto                    62231 non-null  int64         
 4   Comprimento_unit           62231 non-null  int64         
 5   Largura_unit               62231 non-null  int64         
 6   Altura_unit                62231 non-null  int64         
 7   Peso_unit                  62231 non-null  float64       
 8   Peças                      62231 non-null  int64         
 9   Caixa_padrão               62231 non-null  object        
 10  Loja                       62231 non-null  int64         
 11  Classe_de_onda             62231 non-null  int64         
 12  Tipo

In [30]:
df_process_capacitado['chave_loja_buffer_onda'].nunique()

1508

In [31]:
df_sorted_pecas = df_process_capacitado.groupby('chave_loja_buffer_onda')['Peças'].sum()
df_sorted_pecas = df_sorted_pecas.reset_index()
df_sorted_pecas

,chave_loja_buffer_onda,Peças
0,100_CX_4,17
1,100_CX_5,2
2,100_CX_6,43
3,100_CX_7,5
4,100_CX_9,28
...,...,...
1503,9_EN_2,225
1504,9_EN_3,1127
1505,9_EN_4,1464
1506,9_EN_5,32


In [32]:
df_sorted_pecas.query('Peças >= 200').sort_values(by='Peças', ascending=True).head(20)

,chave_loja_buffer_onda,Peças
636,25_EN_2,200
401,143_EN_1,201
635,25_EN_1,203
876,44_CX_3,203
1453,95_EN_4,203
653,27_CX_3,205
1300,82_EN_1,205
461,14_CX_3,205
1361,88_CX_4,207
687,29_EN_3,207


In [33]:
list(df_sorted_pecas.query('Peças >= 400').sort_values(by='Peças', ascending=True).head(20)['chave_loja_buffer_onda'])

['14_EN_4',
 '48_EN_3',
 '34_EN_4',
 '24_EN_3',
 '23_CX_1',
 '44_CX_4',
 '25_EN_3',
 '62_EN_4',
 '21_EN_3',
 '11_EN_3',
 '30_EN_3',
 '17_CX_9',
 '73_EN_4',
 '17_EN_4',
 '9_CX_7',
 '65_EN_4',
 '13_EN_4',
 '20_CX_4',
 '56_EN_4',
 '27_CX_7']

In [34]:
# Selecionar as n lojas maiores de um único buffer
n = 5
lojas_selecionadas_pequenas = list(df_sorted_pecas.query('Peças >= 400').sort_values(by='Peças', ascending=True).head(n)['chave_loja_buffer_onda'])
lojas_selecionadas_pequenas

['14_EN_4', '48_EN_3', '34_EN_4', '24_EN_3', '23_CX_1']

In [35]:
df_process_capacitado.query(f"chave_loja_buffer_onda in ({lojas_selecionadas_pequenas})")

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max
82,82,47,2023-10-24,534996762,364,167,90,0.090,1,CARTON_M,...,777_1.0,23_CX_1,5470920,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170
83,83,47,2023-10-24,552747022,250,100,9,0.108,2,CARTON_M,...,777_3.0,23_CX_1,225000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170
84,84,47,2023-10-24,534996800,365,230,45,0.220,4,CARTON_M,...,777_3.0,23_CX_1,3777750,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170
85,85,47,2023-10-24,539354355,250,100,70,0.200,1,CARTON_M,...,777_95.0,23_CX_1,1750000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170
86,86,47,2023-10-24,539354339,250,100,58,0.175,1,CARTON_M,...,777_95.0,23_CX_1,1450000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61794,61794,1157,2023-11-22,873049031,450,306,23,0.287,4,CARTON_M,...,943_18.0,34_EN_4,3167100,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
61795,61795,8666,2023-11-22,873049102,450,315,20,0.307,2,CARTON_M,...,943_18.0,34_EN_4,2835000,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
61796,61796,9502,2023-11-22,873049049,450,329,25,0.280,3,CARTON_M,...,943_18.0,34_EN_4,3701250,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
61797,61797,10907,2023-11-22,873049057,450,326,25,0.273,1,CARTON_M,...,943_18.0,34_EN_4,3667500,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512


In [36]:
df_process_capacitado.query(f"chave_loja_buffer_onda in ({lojas_selecionadas_pequenas})")['Peças'].sum()

np.int64(2081)

In [37]:
df_process_capacitado["Volume_total"] = df_process_capacitado["Volume_unit"] * df_process_capacitado["Peças"]
df_process_capacitado["Peso_total"] = df_process_capacitado["Peso_unit"] * df_process_capacitado["Peças"]

## Rotas

In [38]:
df_rotas = df_process_capacitado[['Rota', 'Capacidade_rota']].drop_duplicates()
df_rotas

,Rota,Capacidade_rota
0,1,inf
3,3,inf
9,10,2.720000e+10
14,8,4.000000e+10
26,6,2.720000e+10
45,13,2.720000e+10
71,11,2.720000e+10
76,12,1.600000e+10
116,5,2.720000e+10
147,15,4.000000e+10


# 2- Modelagem

## 2.1- HEURÍSTICA - FFD

First Fit Decreasing

In [39]:
import pandas as pd

def first_fit_decreasing(df, bin_restrictions, route_restrictions):
    # Sort items in decreasing order of volume and increasing order of date (FIFO)
    df_sorted = df.copy()
    df_sorted['Inv_Volume_total'] = (1 / df_sorted['Volume_total']).replace(np.inf, 0)
    df_sorted = df_sorted.sort_values(by=['Data_da_ordem_de_produção', 'Inv_Volume_total'], ascending=True)
    
    # Initialize bins list
    bins = []
    leftover_items = pd.DataFrame(columns=df_sorted.columns)  # Initialize leftover items DataFrame
    
    # Flag to track if a split was made
    split_made = True
    items_to_drop = []
    df_sorted = df_sorted.reset_index(drop=True)
    items_to_add = []

    while len(df_sorted) != 0:
        split_made = False
        df_sorted = df_sorted[~df_sorted.index.isin(items_to_drop)]

        # Add needed items to df_sorted
        if len(items_to_add) != 0:
            df_sorted = pd.concat([df_sorted, pd.DataFrame(items_to_add)])
        items_to_add = []
        df_sorted = df_sorted.reset_index(drop=True)
        items_to_drop = []
        
        # Make a copy of the sorted DataFrame
        df_sorted_copy = df_sorted.copy()
        
        # Iterate over items in the copy of the sorted DataFrame
        for index, item in df_sorted_copy.iterrows():
            # Flag to check if the item is placed
            placed = False
            
            # Try to place the item in existing bins
            for bin in bins:
                if (bin['Volume'] + item['Volume_total']) <= bin["vol_limit"] \
                    and (bin['Peso_total'] + item['Peso_total']) <= 23 \
                    and bin['Amt.Items'] + item['Peças'] <= bin["item_limit"] \
                    and item["Loja"] == bin["Loja"] \
                    and item["Rota"] == bin["Rota"] \
                    and item["Tipo_de_buffer"] == bin["Buffer"] \
                    and item["Classe_de_onda"] == bin["Wave"]:
                    
                    bin['Items'].append(index)
                    bin['Volume'] += item['Volume_total']
                    bin['Peso_total'] += item['Peso_total']
                    bin['Amt.Items'] += item['Peças']

                    # Remove the item from the original DataFrame
                    items_to_drop.append(index)
                    placed = True
                    break
            
            # If the item couldn't be placed entirely, reduce it by "inner" means until it fits into a bin
            while not placed and item['Peças'] > item['Inner']:
                # Reduce the item to its inner size
                detached_portion = item.copy()
                detached_portion['Peças'] = item['Inner']
                detached_portion['Volume_total'] = item['Volume_unit'] * item['Inner']
                detached_portion['Peso_total'] = item['Peso_unit'] * item['Inner']
                
                # Update the remaining portion of the item
                item['Peças'] -= item['Inner']
                item['Volume_total'] -= item['Volume_unit'] * item['Inner']
                item['Peso_total'] -= item['Peso_unit'] * item['Inner']
                
                # Concatenate the reduced portion back to the original DataFrame
                items_to_add.append(detached_portion)
                split_made = True
                
                # Try to place the reduced item in existing bins
                for bin in bins:
                    if (bin['Volume'] + item['Volume_total']) <= bin["vol_limit"] \
                        and (bin['Peso_total'] + item['Peso_total']) <= 23 \
                        and bin['Amt.Items'] + item['Peças'] <= bin["item_limit"] \
                        and item["Loja"] == bin["Loja"] \
                        and item["Rota"] == bin["Rota"] \
                        and item["Tipo_de_buffer"] == bin["Buffer"] \
                        and item["Classe_de_onda"] == bin["Wave"]:
                        
                        bin['Items'].append(index)  # Append index of the appended reduced item
                        bin['Volume'] += item['Volume_total']
                        bin['Peso_total'] += item['Peso_total']
                        bin['Amt.Items'] += item['Peças']

                        items_to_drop.append(index) # Remove the reduced item from the original DataFrame
                        placed = True
                        break
            
            # If the item still couldn't be placed, create a new bin
            if not placed:
                # Check if creating a new bin exceeds route capacity
                if len([bin for bin in bins if bin['Rota'] == item["Rota"]]) >= route_restrictions[route_restrictions["Rota"] == item["Rota"]]["Capacidade_rota"].values[0]:
                    leftover_items = leftover_items.append(item)  # Add item to leftover items DataFrame
                else:
                    bins.append({
                        'Items': [index],
                        'Volume': item['Volume_total'],
                        'Peso_total': item['Peso_total'],
                        'Amt.Items': item['Peças'],
                        "Loja": item["Loja"],
                        "Rota": item["Rota"],
                        "Buffer": item["Tipo_de_buffer"],
                        "Wave": item["Classe_de_onda"],
                        "vol_limit": bin_restrictions[bin_restrictions["Tipo_de_buffer"] == item["Tipo_de_buffer"]]["volume_util_caixa"].values[0],
                        "item_limit": bin_restrictions[bin_restrictions["Tipo_de_buffer"] == item["Tipo_de_buffer"]]["Peças_max"].values[0]
                    })
                    items_to_drop.append(index)
                    placed = True
                    break
    
    return bins, leftover_items


In [40]:
def add_item_loja_column(df):
    # Group by 'Loja' and 'Produto' and use cumcount to assign a sequential number starting from 1
    df['item_loja'] = df.groupby(['Loja']).cumcount() + 1
    return df


### 2.1.1- Situação 1

In [41]:
df_ffd_s1 = df_process_capacitado
df_ffd_s1

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,Volume_total,Peso_total
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,3092300,98,98,27778950.0,83393894.4,75538008.0,83393894.4,170,6184600,1.120
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,3321500,98,98,27778950.0,83393894.4,75538008.0,83393894.4,170,9964500,1.800
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,3537200,98,98,27778950.0,83393894.4,75538008.0,83393894.4,170,3537200,0.580
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,233624,37,78,27778950.0,83393894.4,75538008.0,83393894.4,170,233624,0.082
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,775500,37,78,27778950.0,83393894.4,75538008.0,83393894.4,170,7755000,0.920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,3269280,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,6538560,0.426
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,4037285,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,4037285,0.233
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,3231360,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,6462720,0.390
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,4235707,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,4235707,0.323


In [42]:
df_ffd_s1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62231 entries, 0 to 62230
Data columns (total 31 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Id                         62231 non-null  int64         
 1   Ordem_de_produção          62231 non-null  int64         
 2   Data_da_ordem_de_produção  62231 non-null  datetime64[ns]
 3   Produto                    62231 non-null  int64         
 4   Comprimento_unit           62231 non-null  int64         
 5   Largura_unit               62231 non-null  int64         
 6   Altura_unit                62231 non-null  int64         
 7   Peso_unit                  62231 non-null  float64       
 8   Peças                      62231 non-null  int64         
 9   Caixa_padrão               62231 non-null  object        
 10  Loja                       62231 non-null  int64         
 11  Classe_de_onda             62231 non-null  int64         
 12  Tipo

In [43]:
df_ffd_s1['Peças'].sum()

np.int64(219995)

In [44]:
df_ffd_s1['Volume_total'].sum()

np.int64(700949695185)

In [45]:
df_caixas_padrao

,Tipo,Tipo_de_buffer,Comprimento,Largura,Altura,Fator_de_ocupação,Peças_max,Peso_max_(kg),volume_util_caixa
1,Caixa 2,CX,586,436,272,1.2,170,23,83393894.4
5,Caixa 3,EN,728,427,270,0.9,512,23,75538008.0


In [46]:
backlog = df_ffd_s1.copy()
backlog = backlog.reset_index(drop=True)
backlog = add_item_loja_column(backlog)
packed_bins, leftovers = first_fit_decreasing(df_ffd_s1, bin_restrictions=df_caixas_padrao, route_restrictions=df_rotas)
print("Packed bins (new method):")
for i, bin in enumerate(packed_bins):
    print(f"Bin {i+1}: Amt. Items {bin['Amt.Items']} (Total Volume: {bin['Volume']} cubic mm | Total Weight: {bin['Peso_total'] :.4f}kg)")

print(f"\nLeftover items: {leftovers['Peças'].sum()} items")


In [ ]:
from datetime import datetime
import pytz

In [ ]:
SPTz = pytz.timezone("America/Sao_Paulo")
start_time = datetime.now(SPTz)
print(f"Início: {start_time.strftime('%Y-%m-%d %H:%M:%S %Z%z')}")


backlog = df_ffd_s1.copy()
backlog = backlog.reset_index(drop=True)
backlog = add_item_loja_column(backlog)
packed_bins, leftovers = first_fit_decreasing(df_ffd_s1, bin_restrictions=df_caixas_padrao, route_restrictions=df_rotas)
print("Packed bins (new method):")
for i, bin in enumerate(packed_bins):
    print(f"Bin {i+1}: Amt. Items {bin['Amt.Items']} (Total Volume: {bin['Volume']} cubic mm | Total Weight: {bin['Peso_total'] :.4f}kg)")

print(f"\nLeftover items: {leftovers['Peças'].sum()} items")


end_time = datetime.now(SPTz)
duration = end_time - start_time
print(f"Término: {end_time.strftime('%Y-%m-%d %H:%M:%S %Z%z')}")
print(f"Duração: {duration}")

In [ ]:
dgfdgffgd

### 2.1.2- Situação 2

In [46]:
df_ffd_s2 = df_process_capacitado.query("chave_loja_buffer_onda in ('95_EN_4', '25_EN_1', '82_EN_1', '27_CX_3', '14_CX_3')")

df_ffd_s2

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,Volume_total,Peso_total
3419,3419,325,2023-11-07,548143470,304,322,17,0.186,1,CARTON_M,...,1664096,15,78,27778950.0,83393894.4,75538008.0,83393894.4,170,1664096,0.186
3420,3420,325,2023-11-07,548143488,296,280,13,0.155,1,CARTON_M,...,1077440,15,78,27778950.0,83393894.4,75538008.0,83393894.4,170,1077440,0.155
3421,3421,325,2023-11-07,548143496,313,308,13,0.145,3,CARTON_M,...,1253252,15,78,27778950.0,83393894.4,75538008.0,83393894.4,170,3759756,0.435
3422,3422,325,2023-11-07,817850051,418,243,24,0.168,1,CARTON_M,...,2437776,15,78,27778950.0,83393894.4,75538008.0,83393894.4,170,2437776,0.168
3423,3423,325,2023-11-07,548143437,316,194,20,0.140,1,CARTON_M,...,1226080,15,78,27778950.0,83393894.4,75538008.0,83393894.4,170,1226080,0.140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59941,59941,9768,2023-11-21,879747307,320,147,42,0.217,3,CARTON_M,...,1975680,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,5927040,0.651
59942,59942,9431,2023-11-21,879747260,314,155,67,0.290,3,CARTON_M,...,3260890,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,9782670,0.870
61368,61368,3016,2023-11-22,883911579,346,184,18,0.090,5,CARTON_M,...,1145952,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,5729760,0.450
61369,61369,8459,2023-11-22,883911587,349,180,19,0.090,9,CARTON_M,...,1193580,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,10742220,0.810


In [47]:
df_ffd_s2['Peças'].sum()

1021

In [48]:
df_ffd_s2['Volume_total'].sum()

3066413006

In [49]:
backlog = df_ffd_s2.copy()
backlog = backlog.reset_index(drop=True)
backlog = add_item_loja_column(backlog)
packed_bins, leftovers = first_fit_decreasing(df_ffd_s2, bin_restrictions=df_caixas_padrao, route_restrictions=df_rotas)
print("Packed bins (new method):")
for i, bin in enumerate(packed_bins):
    print(f"Bin {i+1}: Amt. Items {bin['Amt.Items']} (Total Volume: {bin['Volume']} cubic mm | Total Weight: {bin['Peso_total'] :.4f}kg)")

print(f"\nLeftover items: {leftovers['Peças'].sum()} items")


Packed bins (new method):
Bin 1: Amt. Items 13 (Total Volume: 75450094 cubic mm | Total Weight: 4.7120kg)
Bin 2: Amt. Items 42 (Total Volume: 83326834 cubic mm | Total Weight: 10.1830kg)
Bin 3: Amt. Items 42 (Total Volume: 83374873 cubic mm | Total Weight: 10.0610kg)
Bin 4: Amt. Items 21 (Total Volume: 75054764 cubic mm | Total Weight: 4.2900kg)
Bin 5: Amt. Items 17 (Total Volume: 74811118 cubic mm | Total Weight: 4.6750kg)
Bin 6: Amt. Items 41 (Total Volume: 75418398 cubic mm | Total Weight: 7.3590kg)
Bin 7: Amt. Items 11 (Total Volume: 75347361 cubic mm | Total Weight: 3.7930kg)
Bin 8: Amt. Items 48 (Total Volume: 75498215 cubic mm | Total Weight: 9.4760kg)
Bin 9: Amt. Items 18 (Total Volume: 74508296 cubic mm | Total Weight: 5.7890kg)
Bin 10: Amt. Items 40 (Total Volume: 82894156 cubic mm | Total Weight: 4.7860kg)
Bin 11: Amt. Items 39 (Total Volume: 83263898 cubic mm | Total Weight: 4.5690kg)
Bin 12: Amt. Items 40 (Total Volume: 83025096 cubic mm | Total Weight: 3.9600kg)
Bin 13: A

### 2.1.3- Situação 3

In [50]:
df_ffd_s3 = df_process_capacitado.query("chave_loja_buffer_onda in ('14_EN_4', '48_EN_3', '34_EN_4', '24_EN_3')")
df_ffd_s3

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,Volume_total,Peso_total
33666,33666,3135,2023-10-18,799851462,532,230,37,0.283,3,CARTON_M,...,4527320,35,78,27778950.0,83393894.4,75538008.0,75538008.0,512,13581960,0.849
33667,33667,10032,2023-10-18,799851471,519,218,38,0.263,3,CARTON_M,...,4299396,35,78,27778950.0,83393894.4,75538008.0,75538008.0,512,12898188,0.789
33668,33668,11487,2023-10-18,799851497,547,198,41,0.247,2,CARTON_M,...,4440546,35,78,27778950.0,83393894.4,75538008.0,75538008.0,512,8881092,0.494
33669,33669,496,2023-10-18,595814344,505,206,40,0.237,3,CARTON_M,...,4161200,35,78,27778950.0,83393894.4,75538008.0,75538008.0,512,12483600,0.711
33670,33670,376,2023-10-18,595814281,494,209,38,0.267,3,CARTON_M,...,3923348,35,78,27778950.0,83393894.4,75538008.0,75538008.0,512,11770044,0.801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61794,61794,1157,2023-11-22,873049031,450,306,23,0.287,4,CARTON_M,...,3167100,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,12668400,1.148
61795,61795,8666,2023-11-22,873049102,450,315,20,0.307,2,CARTON_M,...,2835000,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,5670000,0.614
61796,61796,9502,2023-11-22,873049049,450,329,25,0.280,3,CARTON_M,...,3701250,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,11103750,0.840
61797,61797,10907,2023-11-22,873049057,450,326,25,0.273,1,CARTON_M,...,3667500,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,3667500,0.273


In [51]:
df_ffd_s3['Peças'].sum()

1660

In [52]:
df_ffd_s3['Volume_total'].sum()

7156157465

In [53]:
backlog = df_ffd_s3.copy()
backlog = backlog.reset_index(drop=True)
backlog = add_item_loja_column(backlog)
packed_bins, leftovers = first_fit_decreasing(df_ffd_s3, bin_restrictions=df_caixas_padrao, route_restrictions=df_rotas)
print("Packed bins (new method):")
for i, bin in enumerate(packed_bins):
    print(f"Bin {i+1}: Amt. Items {bin['Amt.Items']} (Total Volume: {bin['Volume']} cubic mm | Total Weight: {bin['Peso_total'] :.4f}kg)")

print(f"\nLeftover items: {leftovers['Peças'].sum()} items")


Packed bins (new method):
Bin 1: Amt. Items 20 (Total Volume: 75351100 cubic mm | Total Weight: 5.0520kg)
Bin 2: Amt. Items 17 (Total Volume: 75385572 cubic mm | Total Weight: 4.4920kg)
Bin 3: Amt. Items 29 (Total Volume: 75111013 cubic mm | Total Weight: 5.0600kg)
Bin 4: Amt. Items 15 (Total Volume: 75475524 cubic mm | Total Weight: 4.8800kg)
Bin 5: Amt. Items 21 (Total Volume: 75244900 cubic mm | Total Weight: 4.3750kg)
Bin 6: Amt. Items 19 (Total Volume: 75148112 cubic mm | Total Weight: 4.5680kg)
Bin 7: Amt. Items 17 (Total Volume: 75517682 cubic mm | Total Weight: 6.7960kg)
Bin 8: Amt. Items 14 (Total Volume: 75536610 cubic mm | Total Weight: 4.2790kg)
Bin 9: Amt. Items 18 (Total Volume: 75519972 cubic mm | Total Weight: 5.6830kg)
Bin 10: Amt. Items 22 (Total Volume: 74907849 cubic mm | Total Weight: 5.2990kg)
Bin 11: Amt. Items 18 (Total Volume: 75405462 cubic mm | Total Weight: 5.3950kg)
Bin 12: Amt. Items 24 (Total Volume: 74361392 cubic mm | Total Weight: 5.4940kg)
Bin 13: Amt

### 2.1.4- Situação 4

In [54]:
df_ffd_s4 = df_process_capacitado.query("chave_loja_buffer_onda in ('44_CX_4', '62_EN_4', '21_EN_3', '11_EN_3', '30_EN_3', '17_CX_9', '73_EN_4')")
df_ffd_s4

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,Volume_total,Peso_total
515,515,1490,2023-11-03,542140454,300,237,32,0.230,3,CARTON_M,...,2275200,19,78,27778950.0,83393894.4,75538008.0,83393894.4,170,6825600,0.690
516,516,1053,2023-11-03,542140454,300,237,32,0.230,2,CARTON_M,...,2275200,19,78,27778950.0,83393894.4,75538008.0,83393894.4,170,4550400,0.460
517,517,1490,2023-11-03,542140446,422,216,23,0.187,3,CARTON_M,...,2096496,19,78,27778950.0,83393894.4,75538008.0,83393894.4,170,6289488,0.561
518,518,1490,2023-11-03,542140462,300,295,19,0.153,1,CARTON_M,...,1681500,19,78,27778950.0,83393894.4,75538008.0,83393894.4,170,1681500,0.153
519,519,1053,2023-11-03,542140462,300,295,19,0.153,1,CARTON_M,...,1681500,19,78,27778950.0,83393894.4,75538008.0,83393894.4,170,1681500,0.153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60096,60096,15,2023-11-22,881488746,491,301,17,0.210,20,CARTON_M,...,2512447,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,50248940,4.200
60097,60097,207,2023-11-22,881488762,454,316,22,0.260,6,CARTON_M,...,3156208,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,18937248,1.560
60098,60098,4833,2023-11-22,881488720,481,294,18,0.240,7,CARTON_M,...,2545452,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,17818164,1.680
60099,60099,6014,2023-11-22,881488738,477,289,19,0.220,18,CARTON_M,...,2619207,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,47145726,3.960


In [55]:
df_ffd_s4['Peças'].sum()

3085

In [56]:
df_ffd_s4['Volume_total'].sum()

12390885045

In [57]:
backlog = df_ffd_s4.copy()
backlog = backlog.reset_index(drop=True)
backlog = add_item_loja_column(backlog)
packed_bins, leftovers = first_fit_decreasing(df_ffd_s4, bin_restrictions=df_caixas_padrao, route_restrictions=df_rotas)
print("Packed bins (new method):")
for i, bin in enumerate(packed_bins):
    print(f"Bin {i+1}: Amt. Items {bin['Amt.Items']} (Total Volume: {bin['Volume']} cubic mm | Total Weight: {bin['Peso_total'] :.4f}kg)")

print(f"\nLeftover items: {leftovers['Peças'].sum()} items")


Packed bins (new method):
Bin 1: Amt. Items 17 (Total Volume: 74891729 cubic mm | Total Weight: 5.8050kg)
Bin 2: Amt. Items 30 (Total Volume: 75238598 cubic mm | Total Weight: 5.5460kg)
Bin 3: Amt. Items 11 (Total Volume: 75494333 cubic mm | Total Weight: 3.9650kg)
Bin 4: Amt. Items 31 (Total Volume: 75434990 cubic mm | Total Weight: 5.5320kg)
Bin 5: Amt. Items 21 (Total Volume: 75328971 cubic mm | Total Weight: 4.5250kg)
Bin 6: Amt. Items 30 (Total Volume: 75394246 cubic mm | Total Weight: 5.3650kg)
Bin 7: Amt. Items 37 (Total Volume: 83362406 cubic mm | Total Weight: 7.7560kg)
Bin 8: Amt. Items 31 (Total Volume: 83341198 cubic mm | Total Weight: 8.3240kg)
Bin 9: Amt. Items 26 (Total Volume: 83004168 cubic mm | Total Weight: 9.9640kg)
Bin 10: Amt. Items 29 (Total Volume: 82995984 cubic mm | Total Weight: 8.0560kg)
Bin 11: Amt. Items 8 (Total Volume: 74305674 cubic mm | Total Weight: 3.3350kg)
Bin 12: Amt. Items 12 (Total Volume: 75219962 cubic mm | Total Weight: 4.8700kg)
Bin 13: Amt.

### 2.1.5- Situação 5

In [58]:
df_ffd_s5 = df_process_capacitado.query("chave_loja_buffer_onda in ('65_EN_4', '9_CX_7', '13_EN_4', '20_CX_4', '31_CX_6')")
df_ffd_s5

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,Volume_total,Peso_total
204,204,1306,2023-10-31,585574931,450,300,24,0.880,4,CARTON_M,...,3240000,22,78,27778950.0,83393894.4,75538008.0,83393894.4,170,12960000,3.520
205,205,1487,2023-10-31,585574893,450,300,40,0.780,1,CARTON_M,...,5400000,22,78,27778950.0,83393894.4,75538008.0,83393894.4,170,5400000,0.780
206,206,1487,2023-10-31,585574906,450,300,25,0.820,1,CARTON_M,...,3375000,22,78,27778950.0,83393894.4,75538008.0,83393894.4,170,3375000,0.820
207,207,1306,2023-10-31,585574893,450,300,40,0.780,2,CARTON_M,...,5400000,22,78,27778950.0,83393894.4,75538008.0,83393894.4,170,10800000,1.560
208,208,1306,2023-10-31,585574906,450,300,25,0.820,1,CARTON_M,...,3375000,22,78,27778950.0,83393894.4,75538008.0,83393894.4,170,3375000,0.820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60700,60700,1157,2023-11-22,873049031,450,306,23,0.287,4,CARTON_M,...,3167100,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,12668400,1.148
60701,60701,5671,2023-11-22,873049065,450,279,23,0.260,2,CARTON_M,...,2887650,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,5775300,0.520
60702,60702,8666,2023-11-22,873049102,450,315,20,0.307,1,CARTON_M,...,2835000,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,2835000,0.307
60703,60703,9502,2023-11-22,873049049,450,329,25,0.280,4,CARTON_M,...,3701250,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,14805000,1.120


In [59]:
df_ffd_s5['Peças'].sum()

2290

In [60]:
df_ffd_s5['Volume_total'].sum()

9012843586

In [61]:
backlog = df_ffd_s5.copy()
backlog = backlog.reset_index(drop=True)
backlog = add_item_loja_column(backlog)
packed_bins, leftovers = first_fit_decreasing(df_ffd_s5, bin_restrictions=df_caixas_padrao, route_restrictions=df_rotas)
print("Packed bins (new method):")
for i, bin in enumerate(packed_bins):
    print(f"Bin {i+1}: Amt. Items {bin['Amt.Items']} (Total Volume: {bin['Volume']} cubic mm | Total Weight: {bin['Peso_total'] :.4f}kg)")

print(f"\nLeftover items: {leftovers['Peças'].sum()} items")


Packed bins (new method):
Bin 1: Amt. Items 24 (Total Volume: 75423966 cubic mm | Total Weight: 8.9610kg)
Bin 2: Amt. Items 16 (Total Volume: 74463282 cubic mm | Total Weight: 4.9650kg)
Bin 3: Amt. Items 21 (Total Volume: 83232651 cubic mm | Total Weight: 11.8690kg)
Bin 4: Amt. Items 18 (Total Volume: 82695185 cubic mm | Total Weight: 10.0000kg)
Bin 5: Amt. Items 7 (Total Volume: 83324243 cubic mm | Total Weight: 3.9400kg)
Bin 6: Amt. Items 61 (Total Volume: 83384039 cubic mm | Total Weight: 22.7900kg)
Bin 7: Amt. Items 15 (Total Volume: 75230716 cubic mm | Total Weight: 4.8600kg)
Bin 8: Amt. Items 17 (Total Volume: 83213547 cubic mm | Total Weight: 9.7040kg)
Bin 9: Amt. Items 8 (Total Volume: 83276416 cubic mm | Total Weight: 7.6400kg)
Bin 10: Amt. Items 11 (Total Volume: 83080109 cubic mm | Total Weight: 6.2200kg)
Bin 11: Amt. Items 13 (Total Volume: 82778372 cubic mm | Total Weight: 5.3500kg)
Bin 12: Amt. Items 16 (Total Volume: 83368180 cubic mm | Total Weight: 10.9890kg)
Bin 13: A

## 2.2- MODELO EXATO

Vamos adicionar uma condição em que o tipo de buffer varia conforme a chave da loja

<p>O peso máximo é mesmo, indenpendente o tipo de buffer. Além disso, nesse momento iremos utilizar o volume padrão para cada buffer</p>

Modelagem matemática 

Dado que temos $n$ tipos de itens $i$ para empacotar, para $k$ lojas, cada um com peso $w_{i k}$, um volume $v_{i k}$, um múltiplo $inner_{i k}$ da quantidade de itens a serem selecionados uma demanda $d_{i k}$, para todo $i \in \{1,...,n\}$ e $k \in \{1,...,r\}$, e um número $m$ de mochilas (caixas) diferentes com capacidade volumétrica $cv_{k}$, uma capacidade de peso $cp$ e uma capacidade em número de peças $np_{k}$, então temos:

$$
\begin{aligned}
\text{minimizar} \quad &\sum_{j = 1}^{m} \sum_{k = 1}^{r} y_{j k} \\
\text{sujeito a} \quad &\sum_{i = 1}^{n} w_{i k} \xi_{i j k}*inner_{i k} \leq cp*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\}\\
&\sum_{i = 1}^{n} v_{i k} \xi_{i j k} *inner_{i k} \leq cv_{k}*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\} \\
&\sum_{i = 1}^{n} \xi_{i j k} *inner_{i k} \leq np_{k}*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\} \\
&\sum_{j = 1}^{m} \xi_{i j k} *inner_{i k} = d_{i k} \quad \forall i \in \{1,...,n\}, k \in \{1,...,r\}\\
&\xi_{i j k} \geq0 \in \mathbb{Z} \quad \forall i \in \{1,...,n\}, j \in \{1,...,m\} , k \in \{1,...,r\} \\
&y_{j k} \in \{0,1\} \quad \forall j \in \{1,...,m\}, k \in \{1,...,r\}
\end{aligned}
$$


 Seja $\quad$ $\xi_{i j k}$= número de itens tipo $i$ adicionados na caixa $j$, para a loja $k$.


In [62]:
%%ampl_eval
reset;

param n;  # número de tipos de itens
param cp; # capacidade de peso de cada caixa
param r;  # número de lojas (rotas)

suffix master IN, binary;
suffix block IN, integer;

set I = 1..n ordered; # tipos de itens
set K = 1..r ordered; # lojas (rotas)
param w {I,K} >= 0; # peso do item tipo i para a loja k
param v {I,K} >= 0; # volume do item tipo i para a loja k
param inner{I,K} >= 0; # múltiplo da quantidade de itens a serem selecionados
param d {I,K} >= 0; # demanda de itens tipo i para a loja k

param cv {K} >= 0; # capacidade volumétrica de cada loja (rota)
param np {K} >= 0; # capacidade em número de peças para cada loja (rota)

param maxVal := max {i in I, k in K} w[i,k];

param maxVal_peso := max {i in I, k in K} w[i,k];
param pecas_caixa:= floor(cp / maxVal_peso);

param soma := sum{i in I, k in K} d[i, k];
param maxbins := ceil(soma / pecas_caixa);

set J = 1..maxbins ordered;

var e {I,J,K} integer >= 0; # número de itens tipo i na caixa j para a chave loja k
var y {J,K} binary; # 1 se a caixa j é usada para a loja k

minimize FO:  sum {j in J, k in K} y[j,k];

s.t. R1_capacidade_mochila_peso {j in J, k in K}:
   sum {i in I} w[i,k] * e[i,j,k] * inner[i,k] <= cp * y[j,k] suffix block j;

s.t. R2_capacidade_mochila_volume {j in J, k in K}:
   sum {i in I} v[i,k] * e[i,j,k] * inner[i,k] <= cv[k] * y[j,k] suffix block j;
   
s.t. R3_capacidade_mochila_pecas {j in J, k in K}:
   sum {i in I} e[i,j,k] * inner[i,k] <= np[k] * y[j,k] suffix block j;

s.t. R4_quantidade_demanda {i in I, k in K}:
   sum {j in J} e[i,j,k]* inner[i,k] = d[i,k] suffix master 1;

### 2.2.1- Situação 1

In [63]:
df_process_situacao1 = df_ffd_s1

df_process_situacao1

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,Volume_total,Peso_total
7129,7129,325,2023-11-07,548143461,309,202,16,0.110,5,CARTON_M,...,998688,15,78,27778950.0,83393894.4,75538008.0,83393894.4,170,4993440,0.550
7130,7130,325,2023-11-07,548143470,304,322,17,0.186,1,CARTON_M,...,1664096,15,78,27778950.0,83393894.4,75538008.0,83393894.4,170,1664096,0.186
7131,7131,325,2023-11-07,548143488,296,280,13,0.155,1,CARTON_M,...,1077440,15,78,27778950.0,83393894.4,75538008.0,83393894.4,170,1077440,0.155
7132,7132,325,2023-11-07,548143496,313,308,13,0.145,1,CARTON_M,...,1253252,15,78,27778950.0,83393894.4,75538008.0,83393894.4,170,1253252,0.145
7133,7133,325,2023-11-07,548143429,317,197,22,0.120,2,CARTON_M,...,1373878,15,78,27778950.0,83393894.4,75538008.0,83393894.4,170,2747756,0.240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61395,61395,3404,2023-11-22,626471971,381,179,44,0.253,1,CARTON_M,...,3000756,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,3000756,0.253
61396,61396,6199,2023-11-22,626472018,415,208,54,0.280,3,CARTON_M,...,4661280,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,13983840,0.840
61397,61397,7945,2023-11-22,626471998,407,188,43,0.217,3,CARTON_M,...,3290188,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,9870564,0.651
61398,61398,8427,2023-11-22,626471955,403,197,43,0.300,3,CARTON_M,...,3413813,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,10241439,0.900


In [64]:
df_process_situacao1['Peças_max'].unique()

array([170, 512], dtype=int64)

In [65]:
df_process_situacao1.columns

Index(['Id', 'Ordem_de_produção', 'Data_da_ordem_de_produção', 'Produto',
       'Comprimento_unit', 'Largura_unit', 'Altura_unit', 'Peso_unit', 'Peças',
       'Caixa_padrão', 'Loja', 'Classe_de_onda', 'Tipo_de_buffer', 'Item_pai',
       'Cor', 'Tamanho', 'Inner', 'Rota', 'Capacidade_rota', 'pai_cor',
       'chave_loja_buffer_onda', 'Volume_unit', 'tempo_espera_dias',
       'prioridade', 'volume_util_caixa_Caixa_1', 'volume_util_caixa_Caixa_2',
       'volume_util_caixa_Caixa_3', 'volume_util_padrao', 'Peças_max',
       'Volume_total', 'Peso_total'],
      dtype='object')

#### 2.2.1.1- Entrada de dados no modelo

In [66]:
# Adicionar uma coluna para identificar cada item dentro de cada chave_loja_buffer_onda
df_process_situacao1['item_loja'] = df_process_situacao1.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peso_unit'

df_matriz_peso = df_process_situacao1.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_peso.columns.name = None

df_matriz_peso

,item_loja,143_EN_1,25_EN_2,44_CX_3
0,1,0.13,0.160,0.110
1,2,0.11,0.220,0.186
2,3,0.13,0.270,0.155
3,4,0.12,0.120,0.145
4,5,0.25,0.113,0.120
...,...,...,...,...
73,74,0.00,0.253,0.000
74,75,0.00,0.280,0.000
75,76,0.00,0.217,0.000
76,77,0.00,0.300,0.000


In [67]:
# Variável avaliada:
variavel_ik = 'Volume_unit'

df_matriz_volume = df_process_situacao1.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_volume.columns.name = None

df_matriz_volume

,item_loja,143_EN_1,25_EN_2,44_CX_3
0,1,2628096.0,859815.0,998688.0
1,2,2872584.0,1426230.0,1664096.0
2,3,3165864.0,2767910.0,1077440.0
3,4,2095510.0,1839120.0,1253252.0
4,5,4732892.0,1952896.0,1373878.0
...,...,...,...,...
73,74,0.0,3000756.0,0.0
74,75,0.0,4661280.0,0.0
75,76,0.0,3290188.0,0.0
76,77,0.0,3413813.0,0.0


In [68]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao1['item_loja'] = df_process_situacao1.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Inner'

df_matriz_inner = df_process_situacao1.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_inner.columns.name = None

df_matriz_inner

,item_loja,143_EN_1,25_EN_2,44_CX_3
0,1,1.0,1.0,1.0
1,2,1.0,1.0,1.0
2,3,1.0,1.0,1.0
3,4,1.0,1.0,1.0
4,5,1.0,1.0,1.0
...,...,...,...,...
73,74,0.0,1.0,0.0
74,75,0.0,1.0,0.0
75,76,0.0,1.0,0.0
76,77,0.0,1.0,0.0


In [69]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao1['item_loja'] = df_process_situacao1.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peças'

df_matriz_pecas = df_process_situacao1.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_pecas.columns.name = None

df_matriz_pecas

,item_loja,143_EN_1,25_EN_2,44_CX_3
0,1,19.0,1.0,5.0
1,2,35.0,1.0,1.0
2,3,9.0,1.0,1.0
3,4,20.0,4.0,1.0
4,5,6.0,7.0,2.0
...,...,...,...,...
73,74,0.0,1.0,0.0
74,75,0.0,3.0,0.0
75,76,0.0,3.0,0.0
76,77,0.0,3.0,0.0


Capacidade da caixa como variável

In [70]:
# Variável avaliada:
variavel_ik = 'Peças_max'

df_matriz_caixa_capacidade_pecas = df_process_situacao1.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_pecas.columns.name = None

df_matriz_caixa_capacidade_pecas

,chave_loja_buffer_onda,Peças_max
0,143_EN_1,512.0
1,25_EN_2,512.0
2,44_CX_3,170.0


In [71]:
# Variável avaliada:
variavel_ik = 'volume_util_padrao'

df_matriz_caixa_capacidade_volume = df_process_situacao1.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_volume.columns.name = None

df_matriz_caixa_capacidade_volume

,chave_loja_buffer_onda,volume_util_padrao
0,143_EN_1,75538008.0
1,25_EN_2,75538008.0
2,44_CX_3,83393894.4


Características das peças

In [72]:
peso =  df_matriz_peso
pecas = df_matriz_pecas  #demanda
volume = df_matriz_volume
inner = df_matriz_inner

Características da caixa

In [73]:
cp = 23
cv = df_matriz_caixa_capacidade_volume
np = df_matriz_caixa_capacidade_pecas

#### Parâmetros

In [74]:
cv['volume_util_padrao'].values

array([75538008.        , 75538008.        , 83393894.39999999])

In [75]:
ampl.param["n"] = len(pecas)   # tamanho da loja que tem mais itens

ampl.param["cp"] = cp
ampl.param["r"] = len(volume.columns) -1

cv_ = cv.drop(columns='chave_loja_buffer_onda')
np_ = np.drop(columns='chave_loja_buffer_onda')

v = volume.drop(columns='item_loja')
w = peso.drop(columns='item_loja')
d= pecas.drop(columns='item_loja')
inner_ = inner.drop(columns='item_loja')

ampl.param["w"] = {(i, k): w.iloc[i - 1][k - 1] for i in range(1, len(w)+1) for k in range(1, len(w.columns)+1)}
ampl.param["v"] = {(i, k): v.iloc[i - 1][k - 1] for i in range(1, len(v)+1) for k in range(1, len(v.columns)+1)}
ampl.param["d"] = {(i, k): d.iloc[i - 1][k - 1] for i in range(1, len(d)+1) for k in range(1, len(d.columns)+1)}
ampl.param["inner"] = {(i, k): inner_.iloc[i - 1][k - 1] for i in range(1, len(inner_)+1) for k in range(1, len(inner_.columns)+1)}

ampl.param["cv"] = {(k): cv_.iloc[k - 1] for k in range(1, len(cv_) + 1)}
ampl.param["np"] = {(k): np_.iloc[k - 1] for k in range(1, len(np_)+1)}

In [76]:
ampl.getValue('soma')

604

In [77]:
ampl.getValue('maxbins')

16

In [78]:
ampl.getValue('pecas_caixa')

40

#### 2.2.1.2- Solvers

##### Solver Highs

In [79]:
ampl.option["solver"] = "highs"
ampl.option["highs_options"] = "outlev=1 timelimit=300"
ampl.solve()

HiGHS 1.7.0:   tech:outlev = 1
  lim:time = 300
Running HiGHS 1.7.0 (git hash: 50670fd): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [6e-02, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 3e+01]
  RHS    [1e+00, 4e+01]
Presolving model
297 rows, 2496 cols, 9936 nonzeros  0s
297 rows, 2496 cols, 9936 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   297 rows
   2496 cols (800 binary, 1696 integer, 0 implied int., 0 continuous)
   9936 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.0s
 R       0       0         0   0.00%   19.28318802     29                33.51%        0      0      0       671   

In [80]:
%%ampl_eval
display d, w, v, inner;

:      d      w        v    inner    :=
1  1   19   0.13    2628100    1
1  2    1   0.16     859815    1
1  3    5   0.11     998688    1
2  1   35   0.11    2872580    1
2  2    1   0.22    1426230    1
2  3    1   0.186   1664100    1
3  1    9   0.13    3165860    1
3  2    1   0.27    2767910    1
3  3    1   0.155   1077440    1
4  1   20   0.12    2095510    1
4  2    4   0.12    1839120    1
4  3    1   0.145   1253250    1
5  1    6   0.25    4732890    1
5  2    7   0.113   1952900    1
5  3    2   0.12    1373880    1
6  1    9   0.23    3645000    1
6  2    4   0.107   1928640    1
6  3    1   0.135   1885670    1
7  1   14   0.22    2962500    1
7  2    4   0.127   1916220    1
7  3    1   0.126   1539650    1
8  1   12   0.2      792810    1
8  2    1   0.218   3741730    1
8  3    1   0.166   2169970    1
9  1    2   0.42    3144150    1
9  2    1   0.1     1915720    1
9  3    1   0.194   2142920    1
10 1    1   0.48    5152000    1
10 2    2   0.13    3721000    1
10 

In [81]:
e_s1_highs = ampl.getVariable('e').getValues().toPandas()
e_s1_highs = e_s1_highs.reset_index()
e_s1_highs.columns = ['item_loja','caixa', 'chave_loja_buffer_onda', 'valor_multiplo']
e_s1_highs = e_s1_highs.query('valor_multiplo >0')
e_s1_highs#.head(20)

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo
2,1,1,3,5
18,1,7,1,17
19,1,7,2,1
36,1,13,1,2
69,2,8,1,14
...,...,...,...,...
3547,74,15,2,1
3568,75,6,2,3
3640,76,14,2,3
3682,77,12,2,3


In [82]:
df_agrupamento_loja1_highs_s1 = e_s1_highs.query('chave_loja_buffer_onda == 1').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja1_highs_s1 = df_agrupamento_loja1_highs_s1.reset_index()
df_agrupamento_loja1_highs_s1.columns.name = None
df_agrupamento_loja1_highs_s1 # as colunas são as caixas

,item_loja,4,7,8,10,11,13,16
0,1,0.0,17.0,0.0,0.0,0.0,2.0,0.0
1,2,0.0,0.0,14.0,0.0,21.0,0.0,0.0
2,3,1.0,0.0,0.0,8.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,20.0,0.0,0.0,0.0
4,5,6.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,0.0,0.0,0.0,0.0,0.0,0.0,9.0
6,7,0.0,0.0,0.0,0.0,0.0,14.0,0.0
7,8,0.0,0.0,11.0,0.0,0.0,1.0,0.0
8,9,0.0,0.0,0.0,2.0,0.0,0.0,0.0
9,10,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [83]:
vetores_por_loja = {}

# Iterar sobre cada grupo (grupo de cada loja)
for nome_loja, grupo in df_process_situacao1.groupby('chave_loja_buffer_onda'):
    # Extrair os IDs do grupo como um vetor
    vetor_ids = grupo['Id'].tolist()
    # Adicionar o vetor à lista de vetores, usando o nome da loja como chave
    vetores_por_loja[nome_loja] = vetor_ids
    # Atribuir o vetor a uma variável com o nome da loja
    exec(f'vetor_{nome_loja} = vetor_ids')

In [84]:
# Lista dos caracteres finais dos nomes dos vetores
char_final = [col for col in w.columns[:3]]

# Criar o dicionário mapeando os caracteres finais aos vetores
vetores = {char: globals()[f'vetor_{char}'] for char in char_final}

In [85]:
df_agrupamento_loja1_highs_s1['Id'] = vetores[f"{w.columns[0]}"]
df_agrupamento_loja1_highs_s1['chave_loja_buffer_onda'] = list(w.columns)[0]

df_agrupamento_loja1_highs_s1

,item_loja,4,7,8,10,11,13,16,Id,chave_loja_buffer_onda
0,1,0.0,17.0,0.0,0.0,0.0,2.0,0.0,35743,143_EN_1
1,2,0.0,0.0,14.0,0.0,21.0,0.0,0.0,35744,143_EN_1
2,3,1.0,0.0,0.0,8.0,0.0,0.0,0.0,35745,143_EN_1
3,4,0.0,0.0,0.0,20.0,0.0,0.0,0.0,35746,143_EN_1
4,5,6.0,0.0,0.0,0.0,0.0,0.0,0.0,35747,143_EN_1
5,6,0.0,0.0,0.0,0.0,0.0,0.0,9.0,35748,143_EN_1
6,7,0.0,0.0,0.0,0.0,0.0,14.0,0.0,35749,143_EN_1
7,8,0.0,0.0,11.0,0.0,0.0,1.0,0.0,35750,143_EN_1
8,9,0.0,0.0,0.0,2.0,0.0,0.0,0.0,35751,143_EN_1
9,10,0.0,0.0,0.0,0.0,0.0,0.0,1.0,35752,143_EN_1


In [86]:
df_agrupamento_loja2_highs_s1 = e_s1_highs.query('chave_loja_buffer_onda == 2').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja2_highs_s1 = df_agrupamento_loja2_highs_s1.reset_index()
df_agrupamento_loja2_highs_s1.columns.name = None
df_agrupamento_loja2_highs_s1['Id'] = vetores[f"{w.columns[1]}"]
df_agrupamento_loja2_highs_s1['chave_loja_buffer_onda'] = list(w.columns)[1]

df_agrupamento_loja2_highs_s1

,item_loja,1,3,6,7,8,12,14,15,Id,chave_loja_buffer_onda
0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,36663,25_EN_2
1,2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,40115,25_EN_2
2,3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,40116,25_EN_2
3,4,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,41031,25_EN_2
4,5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,41032,25_EN_2
...,...,...,...,...,...,...,...,...,...,...,...
73,74,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,61395,25_EN_2
74,75,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,61396,25_EN_2
75,76,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,61397,25_EN_2
76,77,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,61398,25_EN_2


In [87]:
y_s1_highs = ampl.getVariable('y').getValues().toPandas()
y_s1_highs = y_s1_highs.reset_index()
y_s1_highs.columns = ['Caixa','chave_loja_buffer_onda', 'selecao']
y_s1_highs = y_s1_highs.query('selecao>0')

df_caixas_por_loja_highs_s1 = y_s1_highs.pivot_table(index='Caixa', columns='chave_loja_buffer_onda', values='selecao', fill_value=0)
df_caixas_por_loja_highs_s1 = df_caixas_por_loja_highs_s1.reset_index()
df_caixas_por_loja_highs_s1.columns.name = None
df_caixas_por_loja_highs_s1 # as colunas são as lojas

,Caixa,1,2,3
0,1,0.0,1.0,1.0
1,3,0.0,1.0,0.0
2,4,1.0,0.0,0.0
3,6,0.0,1.0,0.0
4,7,1.0,1.0,1.0
5,8,1.0,1.0,0.0
6,9,0.0,0.0,1.0
7,10,1.0,0.0,1.0
8,11,1.0,0.0,0.0
9,12,0.0,1.0,1.0


In [88]:
df_caixas_por_loja_highs_s1.columns = (['caixas'] + list(w.columns)) # os número das colunas são os nomes das lojas
df_caixas_por_loja_highs_s1

,caixas,143_EN_1,25_EN_2,44_CX_3
0,1,0.0,1.0,1.0
1,3,0.0,1.0,0.0
2,4,1.0,0.0,0.0
3,6,0.0,1.0,0.0
4,7,1.0,1.0,1.0
5,8,1.0,1.0,0.0
6,9,0.0,0.0,1.0
7,10,1.0,0.0,1.0
8,11,1.0,0.0,0.0
9,12,0.0,1.0,1.0


##### Caixas

In [89]:
print('Solver Highs')
total_caixas = 0
for col in w.columns:
    print(f'número de caixas para a chave_loja_buffer_onda "{col}": {df_caixas_por_loja_highs_s1[col].sum()}')
    total_caixas += df_caixas_por_loja_highs_s1[col].sum()
print(f'\ntotal de caixas: {total_caixas}')

Solver Highs
número de caixas para a chave_loja_buffer_onda "143_EN_1": 7.0
número de caixas para a chave_loja_buffer_onda "25_EN_2": 8.0
número de caixas para a chave_loja_buffer_onda "44_CX_3": 5.0

total de caixas: 20.0


### 2.2.2- Situação 2

In [90]:
df_process_situacao2 = df_ffd_s2

df_process_situacao2

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,Volume_total,Peso_total
3419,3419,325,2023-11-07,548143470,304,322,17,0.186,1,CARTON_M,...,1664096,15,78,27778950.0,83393894.4,75538008.0,83393894.4,170,1664096,0.186
3420,3420,325,2023-11-07,548143488,296,280,13,0.155,1,CARTON_M,...,1077440,15,78,27778950.0,83393894.4,75538008.0,83393894.4,170,1077440,0.155
3421,3421,325,2023-11-07,548143496,313,308,13,0.145,3,CARTON_M,...,1253252,15,78,27778950.0,83393894.4,75538008.0,83393894.4,170,3759756,0.435
3422,3422,325,2023-11-07,817850051,418,243,24,0.168,1,CARTON_M,...,2437776,15,78,27778950.0,83393894.4,75538008.0,83393894.4,170,2437776,0.168
3423,3423,325,2023-11-07,548143437,316,194,20,0.140,1,CARTON_M,...,1226080,15,78,27778950.0,83393894.4,75538008.0,83393894.4,170,1226080,0.140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59941,59941,9768,2023-11-21,879747307,320,147,42,0.217,3,CARTON_M,...,1975680,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,5927040,0.651
59942,59942,9431,2023-11-21,879747260,314,155,67,0.290,3,CARTON_M,...,3260890,1,78,27778950.0,83393894.4,75538008.0,75538008.0,512,9782670,0.870
61368,61368,3016,2023-11-22,883911579,346,184,18,0.090,5,CARTON_M,...,1145952,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,5729760,0.450
61369,61369,8459,2023-11-22,883911587,349,180,19,0.090,9,CARTON_M,...,1193580,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,10742220,0.810


#### 2.2.2.1- Entrada de dados no modelo

In [91]:
# Adicionar uma coluna para identificar cada item dentro de cada chave_loja_buffer_onda
df_process_situacao2['item_loja'] = df_process_situacao2.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peso_unit'

df_matriz_peso = df_process_situacao2.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_peso.columns.name = None

df_matriz_peso

,item_loja,14_CX_3,25_EN_1,27_CX_3,82_EN_1,95_EN_4
0,1,0.186,0.120,0.186,0.13,0.260
1,2,0.155,0.250,0.155,0.11,0.253
2,3,0.145,0.230,0.140,0.13,0.253
3,4,0.168,0.136,0.156,0.25,0.280
4,5,0.140,0.100,0.135,0.23,0.267
...,...,...,...,...,...,...
75,76,0.000,0.000,0.000,0.00,0.240
76,77,0.000,0.000,0.000,0.00,0.203
77,78,0.000,0.000,0.000,0.00,0.230
78,79,0.000,0.000,0.000,0.00,0.217


In [92]:
# Variável avaliada:
variavel_ik = 'Volume_unit'

df_matriz_volume = df_process_situacao2.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_volume.columns.name = None

df_matriz_volume

,item_loja,14_CX_3,25_EN_1,27_CX_3,82_EN_1,95_EN_4
0,1,1664096.0,2095510.0,1664096.0,2628096.0,3576430.0
1,2,1077440.0,4732892.0,1077440.0,2872584.0,3449628.0
2,3,1253252.0,3645000.0,1226080.0,3165864.0,3449628.0
3,4,2437776.0,625000.0,4084248.0,4732892.0,3536260.0
4,5,1226080.0,1975320.0,1885674.0,3645000.0,3532896.0
...,...,...,...,...,...,...
75,76,0.0,0.0,0.0,0.0,2108970.0
76,77,0.0,0.0,0.0,0.0,3133260.0
77,78,0.0,0.0,0.0,0.0,1725116.0
78,79,0.0,0.0,0.0,0.0,1975680.0


In [93]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao2['item_loja'] = df_process_situacao2.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Inner'

df_matriz_inner = df_process_situacao2.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_inner.columns.name = None

df_matriz_inner

,item_loja,14_CX_3,25_EN_1,27_CX_3,82_EN_1,95_EN_4
0,1,1.0,1.0,1.0,1.0,1.0
1,2,1.0,1.0,1.0,1.0,1.0
2,3,1.0,1.0,1.0,1.0,1.0
3,4,1.0,1.0,1.0,1.0,1.0
4,5,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...
75,76,0.0,0.0,0.0,0.0,1.0
76,77,0.0,0.0,0.0,0.0,1.0
77,78,0.0,0.0,0.0,0.0,1.0
78,79,0.0,0.0,0.0,0.0,1.0


In [94]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao2['item_loja'] = df_process_situacao2.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peças'

df_matriz_pecas = df_process_situacao2.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_pecas.columns.name = None

df_matriz_pecas

,item_loja,14_CX_3,25_EN_1,27_CX_3,82_EN_1,95_EN_4
0,1,1.0,3.0,2.0,2.0,1.0
1,2,1.0,1.0,3.0,8.0,1.0
2,3,3.0,2.0,3.0,4.0,2.0
3,4,1.0,1.0,2.0,1.0,1.0
4,5,1.0,3.0,2.0,3.0,3.0
...,...,...,...,...,...,...
75,76,0.0,0.0,0.0,0.0,4.0
76,77,0.0,0.0,0.0,0.0,1.0
77,78,0.0,0.0,0.0,0.0,5.0
78,79,0.0,0.0,0.0,0.0,3.0


Capacidade da caixa como variável

In [95]:
# Variável avaliada:
variavel_ik = 'Peças_max'

df_matriz_caixa_capacidade_pecas = df_process_situacao2.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_pecas.columns.name = None

df_matriz_caixa_capacidade_pecas

,chave_loja_buffer_onda,Peças_max
0,14_CX_3,170.0
1,25_EN_1,512.0
2,27_CX_3,170.0
3,82_EN_1,512.0
4,95_EN_4,512.0


In [96]:
# Variável avaliada:
variavel_ik = 'volume_util_padrao'

df_matriz_caixa_capacidade_volume = df_process_situacao2.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_volume.columns.name = None

df_matriz_caixa_capacidade_volume

,chave_loja_buffer_onda,volume_util_padrao
0,14_CX_3,83393894.4
1,25_EN_1,75538008.0
2,27_CX_3,83393894.4
3,82_EN_1,75538008.0
4,95_EN_4,75538008.0


In [97]:
peso =  df_matriz_peso
pecas = df_matriz_pecas  #demanda
volume = df_matriz_volume
inner = df_matriz_inner

In [98]:
cp = 23
cv = df_matriz_caixa_capacidade_volume
np = df_matriz_caixa_capacidade_pecas

#### Parâmetros

In [99]:
ampl.param["n"] = len(pecas)   # tamanho da loja que tem mais itens

ampl.param["cp"] = cp
ampl.param["r"] = len(volume.columns) -1

cv_ = cv.drop(columns='chave_loja_buffer_onda')
np_ = np.drop(columns='chave_loja_buffer_onda')

v = volume.drop(columns='item_loja')
w = peso.drop(columns='item_loja')
d= pecas.drop(columns='item_loja')
inner_ = inner.drop(columns='item_loja')

ampl.param["w"] = {(i, k): w.iloc[i - 1][k - 1] for i in range(1, len(w)+1) for k in range(1, len(w.columns)+1)}
ampl.param["v"] = {(i, k): v.iloc[i - 1][k - 1] for i in range(1, len(v)+1) for k in range(1, len(v.columns)+1)}
ampl.param["d"] = {(i, k): d.iloc[i - 1][k - 1] for i in range(1, len(d)+1) for k in range(1, len(d.columns)+1)}
ampl.param["inner"] = {(i, k): inner_.iloc[i - 1][k - 1] for i in range(1, len(inner_)+1) for k in range(1, len(inner_.columns)+1)}

ampl.param["cv"] = {(k): cv_.iloc[k - 1] for k in range(1, len(cv_) + 1)}
ampl.param["np"] = {(k): np_.iloc[k - 1] for k in range(1, len(np_)+1)}

#### 2.2.2.2- Solvers

In [100]:
ampl.option["solver"] = "highs"
ampl.option["highs_options"] = "outlev=1 timelimit=300"
ampl.solve()

HiGHS 1.7.0:   tech:outlev = 1
  lim:time = 300
Running HiGHS 1.7.0 (git hash: 50670fd): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [8e-02, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 2e+01]
  RHS    [1e+00, 2e+01]
Presolving model
Presolve: Infeasible

Solving report
  Status            Infeasible
  Primal bound      inf
  Dual bound        -inf
  Gap               inf
  Solution status   -
  Timing            0.01 (total)
                    0.01 (presolve)
                    0.00 (postsolve)
  Nodes             0
  LP iterations     0 (total)
                    0 (strong br.)
                    0 (separation)
                    0 (heuristics)
Coefficient ranges:
  Matrix [8e-02, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 2e+01]
  RHS    [1e+00, 2e+01]
Presolving model
637 rows, 6399 cols, 25461 nonzeros  0s
637 rows, 6399 cols, 25461 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   637 rows
   6399 c

In [101]:
e_s2_highs = ampl.getVariable('e').getValues().toPandas()
e_s2_highs = e_s2_highs.reset_index()
e_s2_highs.columns = ['item_loja','caixa', 'chave_loja_buffer_onda', 'valor_multiplo']
e_s2_highs = e_s2_highs.query('valor_multiplo >0')
e_s2_highs#.head(20)

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo
10,1,3,1,3.745138e-15
13,1,3,4,1.000000e+00
19,1,4,5,1.000000e+00
27,1,6,3,2.000000e+00
46,1,10,2,1.000000e+00
...,...,...,...,...
10399,78,1,5,5.000000e+00
10421,78,6,2,4.000000e+00
10624,79,19,5,2.000000e+00
10649,79,24,5,1.000000e+00


In [102]:
y_s2_highs = ampl.getVariable('y').getValues().toPandas()
y_s2_highs = y_s2_highs.reset_index()
y_s2_highs.columns = ['Caixa','chave_loja_buffer_onda', 'selecao']
y_s2_highs = y_s2_highs.query('selecao>0')

df_caixas_por_loja_highs_s2 = y_s2_highs.pivot_table(index='Caixa', columns='chave_loja_buffer_onda', values='selecao', fill_value=0)
df_caixas_por_loja_highs_s2 = df_caixas_por_loja_highs_s2.reset_index()
df_caixas_por_loja_highs_s2.columns.name = None
df_caixas_por_loja_highs_s2 # as colunas são as lojas

,Caixa,1,2,3,4,5
0,1,0.0,0.0,0.0,0.0,1.0
1,3,1.0,0.0,1.0,1.0,1.0
2,4,0.0,0.0,0.0,0.0,1.0
3,5,0.0,0.0,0.0,0.0,1.0
4,6,0.0,0.0,1.0,0.0,1.0
5,7,0.0,0.0,0.0,0.0,1.0
6,8,0.0,1.0,0.0,0.0,1.0
7,10,0.0,1.0,0.0,0.0,1.0
8,11,1.0,0.0,1.0,0.0,1.0
9,12,0.0,0.0,1.0,0.0,1.0


In [103]:
df_caixas_por_loja_highs_s2.columns = (['caixas'] + list(w.columns)) # os número das colunas são os nomes das lojas
df_caixas_por_loja_highs_s2

,caixas,14_CX_3,25_EN_1,27_CX_3,82_EN_1,95_EN_4
0,1,0.0,0.0,0.0,0.0,1.0
1,3,1.0,0.0,1.0,1.0,1.0
2,4,0.0,0.0,0.0,0.0,1.0
3,5,0.0,0.0,0.0,0.0,1.0
4,6,0.0,0.0,1.0,0.0,1.0
5,7,0.0,0.0,0.0,0.0,1.0
6,8,0.0,1.0,0.0,0.0,1.0
7,10,0.0,1.0,0.0,0.0,1.0
8,11,1.0,0.0,1.0,0.0,1.0
9,12,0.0,0.0,1.0,0.0,1.0


#### Caixas

In [104]:
print('Solver Highs')
total_caixas = 0
for col in w.columns:
    print(f'número de caixas para a chave_loja_buffer_onda "{col}": {df_caixas_por_loja_highs_s2[col].sum()}')
    total_caixas += df_caixas_por_loja_highs_s2[col].sum()
print(f'\ntotal de caixas: {total_caixas}')

Solver Highs
número de caixas para a chave_loja_buffer_onda "14_CX_3": 5.0
número de caixas para a chave_loja_buffer_onda "25_EN_1": 6.0
número de caixas para a chave_loja_buffer_onda "27_CX_3": 5.0
número de caixas para a chave_loja_buffer_onda "82_EN_1": 6.0
número de caixas para a chave_loja_buffer_onda "95_EN_4": 19.0

total de caixas: 41.0


### 2.2.3- Situação 3

In [105]:
df_process_situacao3 = df_ffd_s3

df_process_situacao3

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,Volume_total,Peso_total
33666,33666,3135,2023-10-18,799851462,532,230,37,0.283,3,CARTON_M,...,4527320,35,78,27778950.0,83393894.4,75538008.0,75538008.0,512,13581960,0.849
33667,33667,10032,2023-10-18,799851471,519,218,38,0.263,3,CARTON_M,...,4299396,35,78,27778950.0,83393894.4,75538008.0,75538008.0,512,12898188,0.789
33668,33668,11487,2023-10-18,799851497,547,198,41,0.247,2,CARTON_M,...,4440546,35,78,27778950.0,83393894.4,75538008.0,75538008.0,512,8881092,0.494
33669,33669,496,2023-10-18,595814344,505,206,40,0.237,3,CARTON_M,...,4161200,35,78,27778950.0,83393894.4,75538008.0,75538008.0,512,12483600,0.711
33670,33670,376,2023-10-18,595814281,494,209,38,0.267,3,CARTON_M,...,3923348,35,78,27778950.0,83393894.4,75538008.0,75538008.0,512,11770044,0.801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61794,61794,1157,2023-11-22,873049031,450,306,23,0.287,4,CARTON_M,...,3167100,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,12668400,1.148
61795,61795,8666,2023-11-22,873049102,450,315,20,0.307,2,CARTON_M,...,2835000,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,5670000,0.614
61796,61796,9502,2023-11-22,873049049,450,329,25,0.280,3,CARTON_M,...,3701250,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,11103750,0.840
61797,61797,10907,2023-11-22,873049057,450,326,25,0.273,1,CARTON_M,...,3667500,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,3667500,0.273


#### 2.2.3.1- Entrada de dados no modelo

In [106]:
# Adicionar uma coluna para identificar cada item dentro de cada chave_loja_buffer_onda
df_process_situacao3['item_loja'] = df_process_situacao3.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peso_unit'

df_matriz_peso = df_process_situacao3.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_peso.columns.name = None

df_matriz_peso

,item_loja,14_EN_4,24_EN_3,34_EN_4,48_EN_3
0,1,0.160,0.283,0.260,0.220
1,2,0.170,0.263,0.300,0.207
2,3,0.330,0.247,0.180,0.180
3,4,0.307,0.237,0.253,0.170
4,5,0.213,0.267,0.253,0.190
...,...,...,...,...,...
147,148,0.000,0.520,0.000,0.000
148,149,0.000,0.500,0.000,0.000
149,150,0.000,0.500,0.000,0.000
150,151,0.000,0.480,0.000,0.000


In [107]:
# Variável avaliada:
variavel_ik = 'Volume_unit'

df_matriz_volume = df_process_situacao3.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_volume.columns.name = None

df_matriz_volume

,item_loja,14_EN_4,24_EN_3,34_EN_4,48_EN_3
0,1,2797520.0,4527320.0,3576430.0,3001320.0
1,2,2507560.0,4299396.0,3907240.0,3228148.0
2,3,4999365.0,4440546.0,2641720.0,2545893.0
3,4,2902500.0,4161200.0,3449628.0,2411200.0
4,5,2984040.0,3923348.0,3449628.0,2504656.0
...,...,...,...,...,...
147,148,0.0,8646506.0,0.0,0.0
148,149,0.0,6992979.0,0.0,0.0
149,150,0.0,7533036.0,0.0,0.0
150,151,0.0,9380525.0,0.0,0.0


In [108]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao3['item_loja'] = df_process_situacao3.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Inner'

df_matriz_inner = df_process_situacao3.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_inner.columns.name = None

df_matriz_inner

,item_loja,14_EN_4,24_EN_3,34_EN_4,48_EN_3
0,1,1.0,1.0,1.0,1.0
1,2,1.0,1.0,1.0,1.0
2,3,1.0,1.0,1.0,1.0
3,4,1.0,1.0,1.0,1.0
4,5,1.0,1.0,1.0,1.0
...,...,...,...,...,...
147,148,0.0,1.0,0.0,0.0
148,149,0.0,1.0,0.0,0.0
149,150,0.0,1.0,0.0,0.0
150,151,0.0,1.0,0.0,0.0


In [109]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao3['item_loja'] = df_process_situacao3.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peças'

df_matriz_pecas = df_process_situacao3.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_pecas.columns.name = None

df_matriz_pecas

,item_loja,14_EN_4,24_EN_3,34_EN_4,48_EN_3
0,1,1.0,3.0,1.0,1.0
1,2,1.0,3.0,1.0,1.0
2,3,1.0,2.0,1.0,1.0
3,4,2.0,3.0,2.0,2.0
4,5,1.0,3.0,1.0,7.0
...,...,...,...,...,...
147,148,0.0,2.0,0.0,0.0
148,149,0.0,3.0,0.0,0.0
149,150,0.0,2.0,0.0,0.0
150,151,0.0,1.0,0.0,0.0


Capacidade da caixa como variável

In [110]:
# Variável avaliada:
variavel_ik = 'Peças_max'

df_matriz_caixa_capacidade_pecas = df_process_situacao3.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_pecas.columns.name = None

df_matriz_caixa_capacidade_pecas

,chave_loja_buffer_onda,Peças_max
0,14_EN_4,512.0
1,24_EN_3,512.0
2,34_EN_4,512.0
3,48_EN_3,512.0


In [111]:
# Variável avaliada:
variavel_ik = 'volume_util_padrao'

df_matriz_caixa_capacidade_volume = df_process_situacao3.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_volume.columns.name = None

df_matriz_caixa_capacidade_volume

,chave_loja_buffer_onda,volume_util_padrao
0,14_EN_4,75538008.0
1,24_EN_3,75538008.0
2,34_EN_4,75538008.0
3,48_EN_3,75538008.0


In [112]:
peso =  df_matriz_peso
pecas = df_matriz_pecas  #demanda
volume = df_matriz_volume
inner = df_matriz_inner

In [113]:
cp = 23
cv = df_matriz_caixa_capacidade_volume
np = df_matriz_caixa_capacidade_pecas

#### Parâmetros

In [114]:
ampl.param["n"] = len(pecas)   # tamanho da loja que tem mais itens

ampl.param["cp"] = cp
ampl.param["r"] = len(volume.columns) -1

cv_ = cv.drop(columns='chave_loja_buffer_onda')
np_ = np.drop(columns='chave_loja_buffer_onda')

v = volume.drop(columns='item_loja')
w = peso.drop(columns='item_loja')
d= pecas.drop(columns='item_loja')
inner_ = inner.drop(columns='item_loja')

ampl.param["w"] = {(i, k): w.iloc[i - 1][k - 1] for i in range(1, len(w)+1) for k in range(1, len(w.columns)+1)}
ampl.param["v"] = {(i, k): v.iloc[i - 1][k - 1] for i in range(1, len(v)+1) for k in range(1, len(v.columns)+1)}
ampl.param["d"] = {(i, k): d.iloc[i - 1][k - 1] for i in range(1, len(d)+1) for k in range(1, len(d.columns)+1)}
ampl.param["inner"] = {(i, k): inner_.iloc[i - 1][k - 1] for i in range(1, len(inner_)+1) for k in range(1, len(inner_.columns)+1)}

ampl.param["cv"] = {(k): cv_.iloc[k - 1] for k in range(1, len(cv_) + 1)}
ampl.param["np"] = {(k): np_.iloc[k - 1] for k in range(1, len(np_)+1)}

#### 2.2.3.2- Solvers

In [115]:
ampl.option["solver"] = "highs"
ampl.option["highs_options"] = "outlev=1 timelimit=600"
ampl.solve()

HiGHS 1.7.0:   tech:outlev = 1
  lim:time = 600
Running HiGHS 1.7.0 (git hash: 50670fd): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [7e-02, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [4e-16, 2e+01]
  RHS    [1e+00, 2e+01]
Presolving model
Presolve: Infeasible

Solving report
  Status            Infeasible
  Primal bound      inf
  Dual bound        -inf
  Gap               inf
  Solution status   -
  Timing            0.02 (total)
                    0.01 (presolve)
                    0.00 (postsolve)
  Nodes             0
  LP iterations     0 (total)
                    0 (strong br.)
                    0 (separation)
                    0 (heuristics)
Coefficient ranges:
  Matrix [7e-02, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 2e+01]
  RHS    [1e+00, 2e+01]
Presolving model
1262 rows, 32760 cols, 130800 nonzeros  0s
1262 rows, 32760 cols, 130800 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   1262 rows
  

In [116]:
e_s3_highs = ampl.getVariable('e').getValues().toPandas()
e_s3_highs = e_s3_highs.reset_index()
e_s3_highs.columns = ['item_loja','caixa', 'chave_loja_buffer_onda', 'valor_multiplo']
e_s3_highs = e_s3_highs.query('valor_multiplo >0')
e_s3_highs#.head(20)

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo
23,1,6,4,1
49,1,13,2,3
84,1,22,1,1
102,1,26,3,1
283,2,11,4,1
...,...,...,...,...
35689,149,43,2,3
35909,150,38,2,1
35961,150,51,2,1
36145,151,37,2,1


In [117]:
y_s3_highs = ampl.getVariable('y').getValues().toPandas()
y_s3_highs = y_s3_highs.reset_index()
y_s3_highs.columns = ['Caixa','chave_loja_buffer_onda', 'selecao']
y_s3_highs = y_s3_highs.query('selecao>0')

df_caixas_por_loja_highs_s3 = y_s3_highs.pivot_table(index='Caixa', columns='chave_loja_buffer_onda', values='selecao', fill_value=0)
df_caixas_por_loja_highs_s3 = df_caixas_por_loja_highs_s3.reset_index()
df_caixas_por_loja_highs_s3.columns.name = None
df_caixas_por_loja_highs_s3 # as colunas são as lojas

,Caixa,1,2,3,4
0,1,0.0,1.0,1.0,1.0
1,2,0.0,1.0,1.0,0.0
2,3,1.0,0.0,1.0,1.0
3,4,0.0,1.0,1.0,0.0
4,5,0.0,0.0,1.0,0.0
5,6,0.0,0.0,1.0,1.0
6,8,0.0,1.0,0.0,0.0
7,10,0.0,1.0,0.0,1.0
8,11,1.0,0.0,0.0,1.0
9,12,1.0,0.0,0.0,0.0


In [118]:
df_caixas_por_loja_highs_s3.columns = (['caixas'] + list(w.columns)) # os número das colunas são os nomes das lojas
df_caixas_por_loja_highs_s3

,caixas,14_EN_4,24_EN_3,34_EN_4,48_EN_3
0,1,0.0,1.0,1.0,1.0
1,2,0.0,1.0,1.0,0.0
2,3,1.0,0.0,1.0,1.0
3,4,0.0,1.0,1.0,0.0
4,5,0.0,0.0,1.0,0.0
5,6,0.0,0.0,1.0,1.0
6,8,0.0,1.0,0.0,0.0
7,10,0.0,1.0,0.0,1.0
8,11,1.0,0.0,0.0,1.0
9,12,1.0,0.0,0.0,0.0


#### Caixas

In [119]:
print('Solver Highs')
total_caixas = 0
for col in w.columns:
    print(f'número de caixas para a chave_loja_buffer_onda "{col}": {df_caixas_por_loja_highs_s3[col].sum()}')
    total_caixas += df_caixas_por_loja_highs_s3[col].sum()
print(f'\ntotal de caixas: {total_caixas}')

Solver Highs
número de caixas para a chave_loja_buffer_onda "14_EN_4": 26.0
número de caixas para a chave_loja_buffer_onda "24_EN_3": 23.0
número de caixas para a chave_loja_buffer_onda "34_EN_4": 30.0
número de caixas para a chave_loja_buffer_onda "48_EN_3": 17.0

total de caixas: 96.0


### 2.2.4- Situação 4

In [63]:
df_process_situacao4 = df_ffd_s4

df_process_situacao4

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,Volume_total,Peso_total
515,515,1490,2023-11-03,542140454,300,237,32,0.230,3,CARTON_M,...,2275200,19,78,27778950.0,83393894.4,75538008.0,83393894.4,170,6825600,0.690
516,516,1053,2023-11-03,542140454,300,237,32,0.230,2,CARTON_M,...,2275200,19,78,27778950.0,83393894.4,75538008.0,83393894.4,170,4550400,0.460
517,517,1490,2023-11-03,542140446,422,216,23,0.187,3,CARTON_M,...,2096496,19,78,27778950.0,83393894.4,75538008.0,83393894.4,170,6289488,0.561
518,518,1490,2023-11-03,542140462,300,295,19,0.153,1,CARTON_M,...,1681500,19,78,27778950.0,83393894.4,75538008.0,83393894.4,170,1681500,0.153
519,519,1053,2023-11-03,542140462,300,295,19,0.153,1,CARTON_M,...,1681500,19,78,27778950.0,83393894.4,75538008.0,83393894.4,170,1681500,0.153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60096,60096,15,2023-11-22,881488746,491,301,17,0.210,20,CARTON_M,...,2512447,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,50248940,4.200
60097,60097,207,2023-11-22,881488762,454,316,22,0.260,6,CARTON_M,...,3156208,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,18937248,1.560
60098,60098,4833,2023-11-22,881488720,481,294,18,0.240,7,CARTON_M,...,2545452,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,17818164,1.680
60099,60099,6014,2023-11-22,881488738,477,289,19,0.220,18,CARTON_M,...,2619207,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,47145726,3.960


#### 2.2.4.1- Entrada de dados no modelo

In [64]:
# Adicionar uma coluna para identificar cada item dentro de cada chave_loja_buffer_onda
df_process_situacao4['item_loja'] = df_process_situacao4.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peso_unit'

df_matriz_peso = df_process_situacao4.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_peso.columns.name = None

df_matriz_peso

,item_loja,11_EN_3,17_CX_9,21_EN_3,30_EN_3,44_CX_4,62_EN_4,73_EN_4
0,1,0.220,0.230,0.200,0.240,0.78,0.30,0.360
1,2,0.207,0.230,0.247,0.213,0.52,0.28,0.325
2,3,0.140,0.187,0.207,0.164,0.32,0.28,0.360
3,4,0.167,0.153,0.167,0.156,0.29,0.66,0.300
4,5,0.240,0.153,0.167,0.185,0.49,0.57,0.280
...,...,...,...,...,...,...,...,...
164,165,0.000,0.000,0.500,0.000,0.00,0.00,0.000
165,166,0.000,0.000,0.460,0.000,0.00,0.00,0.000
166,167,0.000,0.000,0.440,0.000,0.00,0.00,0.000
167,168,0.000,0.000,0.410,0.000,0.00,0.00,0.000


In [65]:
# Variável avaliada:
variavel_ik = 'Volume_unit'

df_matriz_volume = df_process_situacao4.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_volume.columns.name = None

df_matriz_volume

,item_loja,11_EN_3,17_CX_9,21_EN_3,30_EN_3,44_CX_4,62_EN_4,73_EN_4
0,1,3001320.0,2275200.0,1633230.0,2864650.0,6219275.0,4237839.0,3129385.0
1,2,3228148.0,2275200.0,3332070.0,2456160.0,6976800.0,3243240.0,3448182.0
2,3,2414425.0,2096496.0,3228148.0,1207720.0,5422560.0,4013658.0,3038868.0
3,4,2741220.0,1681500.0,2741220.0,1212300.0,4697784.0,2888550.0,4237839.0
4,5,2864650.0,1681500.0,2741220.0,2637180.0,3432780.0,2781900.0,3243240.0
...,...,...,...,...,...,...,...,...
164,165,0.0,0.0,5348480.0,0.0,0.0,0.0,0.0
165,166,0.0,0.0,4516050.0,0.0,0.0,0.0,0.0
166,167,0.0,0.0,4362732.0,0.0,0.0,0.0,0.0
167,168,0.0,0.0,4469256.0,0.0,0.0,0.0,0.0


In [66]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao4['item_loja'] = df_process_situacao4.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Inner'

df_matriz_inner = df_process_situacao4.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_inner.columns.name = None

df_matriz_inner

,item_loja,11_EN_3,17_CX_9,21_EN_3,30_EN_3,44_CX_4,62_EN_4,73_EN_4
0,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,2,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,3,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,4,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,5,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...
164,165,0.0,0.0,1.0,0.0,0.0,0.0,0.0
165,166,0.0,0.0,1.0,0.0,0.0,0.0,0.0
166,167,0.0,0.0,1.0,0.0,0.0,0.0,0.0
167,168,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [67]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao4['item_loja'] = df_process_situacao4.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peças'

df_matriz_pecas = df_process_situacao4.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_pecas.columns.name = None

df_matriz_pecas

,item_loja,11_EN_3,17_CX_9,21_EN_3,30_EN_3,44_CX_4,62_EN_4,73_EN_4
0,1,1.0,3.0,1.0,1.0,1.0,2.0,5.0
1,2,1.0,2.0,1.0,1.0,1.0,1.0,3.0
2,3,1.0,3.0,1.0,1.0,2.0,1.0,9.0
3,4,2.0,1.0,1.0,1.0,1.0,2.0,2.0
4,5,1.0,1.0,3.0,3.0,1.0,2.0,3.0
...,...,...,...,...,...,...,...,...
164,165,0.0,0.0,1.0,0.0,0.0,0.0,0.0
165,166,0.0,0.0,2.0,0.0,0.0,0.0,0.0
166,167,0.0,0.0,5.0,0.0,0.0,0.0,0.0
167,168,0.0,0.0,2.0,0.0,0.0,0.0,0.0


Capacidade da caixa como variável

In [68]:
# Variável avaliada:
variavel_ik = 'Peças_max'

df_matriz_caixa_capacidade_pecas = df_process_situacao4.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_pecas.columns.name = None

df_matriz_caixa_capacidade_pecas

,chave_loja_buffer_onda,Peças_max
0,11_EN_3,512.0
1,17_CX_9,170.0
2,21_EN_3,512.0
3,30_EN_3,512.0
4,44_CX_4,170.0
5,62_EN_4,512.0
6,73_EN_4,512.0


In [69]:
# Variável avaliada:
variavel_ik = 'volume_util_padrao'

df_matriz_caixa_capacidade_volume = df_process_situacao4.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_volume.columns.name = None

df_matriz_caixa_capacidade_volume

,chave_loja_buffer_onda,volume_util_padrao
0,11_EN_3,75538008.0
1,17_CX_9,83393894.4
2,21_EN_3,75538008.0
3,30_EN_3,75538008.0
4,44_CX_4,83393894.4
5,62_EN_4,75538008.0
6,73_EN_4,75538008.0


In [70]:
peso =  df_matriz_peso
pecas = df_matriz_pecas  #demanda
volume = df_matriz_volume
inner = df_matriz_inner

In [71]:
cp = 23
cv = df_matriz_caixa_capacidade_volume
np = df_matriz_caixa_capacidade_pecas

#### Parâmetros

In [72]:
ampl.param["n"] = len(pecas)   # tamanho da loja que tem mais itens

ampl.param["cp"] = cp
ampl.param["r"] = len(volume.columns) -1

cv_ = cv.drop(columns='chave_loja_buffer_onda')
np_ = np.drop(columns='chave_loja_buffer_onda')

v = volume.drop(columns='item_loja')
w = peso.drop(columns='item_loja')
d= pecas.drop(columns='item_loja')
inner_ = inner.drop(columns='item_loja')

ampl.param["w"] = {(i, k): w.iloc[i - 1][k - 1] for i in range(1, len(w)+1) for k in range(1, len(w.columns)+1)}
ampl.param["v"] = {(i, k): v.iloc[i - 1][k - 1] for i in range(1, len(v)+1) for k in range(1, len(v.columns)+1)}
ampl.param["d"] = {(i, k): d.iloc[i - 1][k - 1] for i in range(1, len(d)+1) for k in range(1, len(d.columns)+1)}
ampl.param["inner"] = {(i, k): inner_.iloc[i - 1][k - 1] for i in range(1, len(inner_)+1) for k in range(1, len(inner_.columns)+1)}

ampl.param["cv"] = {(k): cv_.iloc[k - 1] for k in range(1, len(cv_) + 1)}
ampl.param["np"] = {(k): np_.iloc[k - 1] for k in range(1, len(np_)+1)}

#### 2.2.4.2- Solvers

In [73]:
ampl.option["solver"] = "highs"
ampl.option["highs_options"] = "outlev=1"  # timelimit=900
ampl.solve()

HiGHS 1.7.0:   tech:outlev = 1
Running HiGHS 1.7.0 (git hash: 50670fd): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [5e-02, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 2e+01]
  RHS    [1e+00, 2e+01]
Presolving model
3429 rows, 111503 cols, 445179 nonzeros  0s
3429 rows, 111503 cols, 445179 nonzeros  2s
Objective function is integral with scale 1

Solving MIP model with:
   3429 rows
   111503 cols (38437 binary, 73066 integer, 0 implied int., 0 continuous)
   445179 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     2.6s
 R       0       0         0   0.00%   160.9262693     257               37.38%        0      0      0     15132  

In [74]:
e_s4_highs = ampl.getVariable('e').getValues().toPandas()
e_s4_highs = e_s4_highs.reset_index()
e_s4_highs.columns = ['item_loja','caixa', 'chave_loja_buffer_onda', 'valor_multiplo']
e_s4_highs = e_s4_highs.query('valor_multiplo >0')
e_s4_highs#.head(20)

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo


In [75]:
y_s4_highs = ampl.getVariable('y').getValues().toPandas()
y_s4_highs = y_s4_highs.reset_index()
y_s4_highs.columns = ['Caixa','chave_loja_buffer_onda', 'selecao']
y_s4_highs = y_s4_highs.query('selecao>0')

df_caixas_por_loja_highs_s4 = y_s4_highs.pivot_table(index='Caixa', columns='chave_loja_buffer_onda', values='selecao', fill_value=0)
df_caixas_por_loja_highs_s4 = df_caixas_por_loja_highs_s4.reset_index()
df_caixas_por_loja_highs_s4.columns.name = None
df_caixas_por_loja_highs_s4 # as colunas são as lojas

,Caixa


In [76]:
df_caixas_por_loja_highs_s4.columns = (['caixas'] + list(w.columns)) # os número das colunas são os nomes das lojas
df_caixas_por_loja_highs_s4

ValueError: Length mismatch: Expected axis has 1 elements, new values have 8 elements

#### Caixas

In [ ]:
print('Solver Highs')
total_caixas = 0
for col in w.columns:
    print(f'número de caixas para a chave_loja_buffer_onda "{col}": {df_caixas_por_loja_highs_s4[col].sum()}')
    total_caixas += df_caixas_por_loja_highs_s4[col].sum()
print(f'\ntotal de caixas: {total_caixas}')

Solver Highs
número de caixas para a chave_loja_buffer_onda "11_EN_3": 17.0
número de caixas para a chave_loja_buffer_onda "17_CX_9": 14.0
número de caixas para a chave_loja_buffer_onda "21_EN_3": 21.0
número de caixas para a chave_loja_buffer_onda "30_EN_3": 25.0
número de caixas para a chave_loja_buffer_onda "44_CX_4": 17.0
número de caixas para a chave_loja_buffer_onda "62_EN_4": 33.0
número de caixas para a chave_loja_buffer_onda "73_EN_4": 38.0

total de caixas: 165.0


In [ ]:
f

### 2.2.5- Situação 5

In [ ]:
df_process_situacao5 = df_ffd_s5

df_process_situacao5

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,Volume_total,Peso_total
204,204,1306,2023-10-31,585574931,450,300,24,0.880,4,CARTON_M,...,3240000,22,78,27778950.0,83393894.4,75538008.0,83393894.4,170,12960000,3.520
205,205,1487,2023-10-31,585574893,450,300,40,0.780,1,CARTON_M,...,5400000,22,78,27778950.0,83393894.4,75538008.0,83393894.4,170,5400000,0.780
206,206,1487,2023-10-31,585574906,450,300,25,0.820,1,CARTON_M,...,3375000,22,78,27778950.0,83393894.4,75538008.0,83393894.4,170,3375000,0.820
207,207,1306,2023-10-31,585574893,450,300,40,0.780,2,CARTON_M,...,5400000,22,78,27778950.0,83393894.4,75538008.0,83393894.4,170,10800000,1.560
208,208,1306,2023-10-31,585574906,450,300,25,0.820,1,CARTON_M,...,3375000,22,78,27778950.0,83393894.4,75538008.0,83393894.4,170,3375000,0.820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60700,60700,1157,2023-11-22,873049031,450,306,23,0.287,4,CARTON_M,...,3167100,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,12668400,1.148
60701,60701,5671,2023-11-22,873049065,450,279,23,0.260,2,CARTON_M,...,2887650,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,5775300,0.520
60702,60702,8666,2023-11-22,873049102,450,315,20,0.307,1,CARTON_M,...,2835000,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,2835000,0.307
60703,60703,9502,2023-11-22,873049049,450,329,25,0.280,4,CARTON_M,...,3701250,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512,14805000,1.120


#### 2.2.5.1- Entrada de dados no modelo

In [ ]:
# Adicionar uma coluna para identificar cada item dentro de cada chave_loja_buffer_onda
df_process_situacao5['item_loja'] = df_process_situacao5.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peso_unit'

df_matriz_peso = df_process_situacao5.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_peso.columns.name = None

df_matriz_peso

,item_loja,13_EN_4,20_CX_4,31_CX_6,65_EN_4,9_CX_7
0,1,0.323,0.88,0.240,0.300,0.26
1,2,0.303,0.78,0.240,0.180,0.24
2,3,0.295,0.82,0.275,0.180,0.26
3,4,0.328,0.78,0.390,0.253,0.22
4,5,0.310,0.82,0.390,0.253,0.94
...,...,...,...,...,...,...
207,208,0.273,0.86,0.000,0.000,0.00
208,209,0.000,0.88,0.000,0.000,0.00
209,210,0.000,0.86,0.000,0.000,0.00
210,211,0.000,0.86,0.000,0.000,0.00


In [ ]:
# Variável avaliada:
variavel_ik = 'Volume_unit'

df_matriz_volume = df_process_situacao5.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_volume.columns.name = None

df_matriz_volume

,item_loja,13_EN_4,20_CX_4,31_CX_6,65_EN_4,9_CX_7
0,1,3553770.0,3240000.0,1506720.0,3907240.0,2933673.0
1,2,3303008.0,5400000.0,1506720.0,2641720.0,2508520.0
2,3,3297150.0,3375000.0,246960.0,2641720.0,2885440.0
3,4,3583140.0,5400000.0,1634640.0,3449628.0,1996400.0
4,5,3428352.0,3375000.0,1634640.0,3449628.0,6997046.0
...,...,...,...,...,...,...
207,208,3667500.0,6095603.0,0.0,0.0,0.0
208,209,0.0,5888160.0,0.0,0.0,0.0
209,210,0.0,5136992.0,0.0,0.0,0.0
210,211,0.0,7524000.0,0.0,0.0,0.0


In [ ]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao5['item_loja'] = df_process_situacao5.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Inner'

df_matriz_inner = df_process_situacao5.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_inner.columns.name = None

df_matriz_inner

,item_loja,13_EN_4,20_CX_4,31_CX_6,65_EN_4,9_CX_7
0,1,1.0,1.0,1.0,1.0,1.0
1,2,1.0,1.0,1.0,1.0,1.0
2,3,1.0,1.0,1.0,1.0,1.0
3,4,1.0,1.0,1.0,1.0,1.0
4,5,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...
207,208,1.0,1.0,0.0,0.0,0.0
208,209,0.0,1.0,0.0,0.0,0.0
209,210,0.0,1.0,0.0,0.0,0.0
210,211,0.0,1.0,0.0,0.0,0.0


In [ ]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao5['item_loja'] = df_process_situacao5.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peças'

df_matriz_pecas = df_process_situacao5.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_pecas.columns.name = None

df_matriz_pecas

,item_loja,13_EN_4,20_CX_4,31_CX_6,65_EN_4,9_CX_7
0,1,2.0,4.0,2.0,1.0,3.0
1,2,2.0,1.0,1.0,1.0,6.0
2,3,1.0,1.0,1.0,2.0,8.0
3,4,1.0,2.0,14.0,2.0,2.0
4,5,2.0,1.0,20.0,3.0,2.0
...,...,...,...,...,...,...
207,208,4.0,2.0,0.0,0.0,0.0
208,209,0.0,1.0,0.0,0.0,0.0
209,210,0.0,1.0,0.0,0.0,0.0
210,211,0.0,2.0,0.0,0.0,0.0


Capacidade da caixa como variável

In [ ]:
# Variável avaliada:
variavel_ik = 'Peças_max'

df_matriz_caixa_capacidade_pecas = df_process_situacao5.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_pecas.columns.name = None

df_matriz_caixa_capacidade_pecas

,chave_loja_buffer_onda,Peças_max
0,13_EN_4,512.0
1,20_CX_4,170.0
2,31_CX_6,170.0
3,65_EN_4,512.0
4,9_CX_7,170.0


In [ ]:
# Variável avaliada:
variavel_ik = 'volume_util_padrao'

df_matriz_caixa_capacidade_volume = df_process_situacao5.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_volume.columns.name = None

df_matriz_caixa_capacidade_volume

,chave_loja_buffer_onda,volume_util_padrao
0,13_EN_4,75538008.0
1,20_CX_4,83393894.4
2,31_CX_6,83393894.4
3,65_EN_4,75538008.0
4,9_CX_7,83393894.4


In [ ]:
peso =  df_matriz_peso
pecas = df_matriz_pecas  #demanda
volume = df_matriz_volume
inner = df_matriz_inner

In [ ]:
cp = 23
cv = df_matriz_caixa_capacidade_volume
np = df_matriz_caixa_capacidade_pecas

#### Parâmetros

In [ ]:
ampl.param["n"] = len(pecas)   # tamanho da loja que tem mais itens

ampl.param["cp"] = cp
ampl.param["r"] = len(volume.columns) -1

cv_ = cv.drop(columns='chave_loja_buffer_onda')
np_ = np.drop(columns='chave_loja_buffer_onda')

v = volume.drop(columns='item_loja')
w = peso.drop(columns='item_loja')
d= pecas.drop(columns='item_loja')
inner_ = inner.drop(columns='item_loja')

ampl.param["w"] = {(i, k): w.iloc[i - 1][k - 1] for i in range(1, len(w)+1) for k in range(1, len(w.columns)+1)}
ampl.param["v"] = {(i, k): v.iloc[i - 1][k - 1] for i in range(1, len(v)+1) for k in range(1, len(v.columns)+1)}
ampl.param["d"] = {(i, k): d.iloc[i - 1][k - 1] for i in range(1, len(d)+1) for k in range(1, len(d.columns)+1)}
ampl.param["inner"] = {(i, k): inner_.iloc[i - 1][k - 1] for i in range(1, len(inner_)+1) for k in range(1, len(inner_.columns)+1)}

ampl.param["cv"] = {(k): cv_.iloc[k - 1] for k in range(1, len(cv_) + 1)}
ampl.param["np"] = {(k): np_.iloc[k - 1] for k in range(1, len(np_)+1)}

#### 2.2.5.2- Solvers

In [ ]:
ampl.option["solver"] = "highs"
ampl.option["highs_options"] = "outlev=1 timelimit=600"
ampl.solve()

HiGHS 1.7.0:   tech:outlev = 1
  lim:time = 600
Running HiGHS 1.7.0 (git hash: 50670fd): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [2e-02, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [1e-17, 9e+00]
  RHS    [1e+00, 6e+01]
Presolving model
Presolve: Infeasible

Solving report
  Status            Infeasible
  Primal bound      inf
  Dual bound        -inf
  Gap               inf
  Solution status   -
  Timing            0.03 (total)
                    0.02 (presolve)
                    0.00 (postsolve)
  Nodes             0
  LP iterations     0 (total)
                    0 (strong br.)
                    0 (separation)
                    0 (heuristics)
Coefficient ranges:
  Matrix [2e-02, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 6e+01]
  RHS    [1e+00, 6e+01]
Presolving model
2501 rows, 89815 cols, 358685 nonzeros  0s
2501 rows, 89815 cols, 358685 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   2501 rows
  

In [ ]:
e_s5_highs = ampl.getVariable('e').getValues().toPandas()
e_s5_highs = e_s5_highs.reset_index()
e_s5_highs.columns = ['item_loja','caixa', 'chave_loja_buffer_onda', 'valor_multiplo']
e_s5_highs = e_s5_highs.query('valor_multiplo >0')
e_s5_highs#.head(20)

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo
36,1,8,2,8.401265e-17
86,1,18,2,4.000000e+00
284,1,57,5,2.000000e+00
393,1,79,4,1.000000e+00
394,1,79,5,1.000000e+00
...,...,...,...,...
120066,209,94,2,1.000000e+00
120736,210,113,2,1.000000e+00
121121,211,75,2,2.000000e+00
121411,212,18,2,1.000000e+00


In [ ]:
y_s5_highs = ampl.getVariable('y').getValues().toPandas()
y_s5_highs = y_s5_highs.reset_index()
y_s5_highs.columns = ['Caixa','chave_loja_buffer_onda', 'selecao']
y_s5_highs = y_s5_highs.query('selecao>0')

df_caixas_por_loja_highs_s5 = y_s5_highs.pivot_table(index='Caixa', columns='chave_loja_buffer_onda', values='selecao', fill_value=0)
df_caixas_por_loja_highs_s5 = df_caixas_por_loja_highs_s5.reset_index()
df_caixas_por_loja_highs_s5.columns.name = None
df_caixas_por_loja_highs_s5 # as colunas são as lojas

,Caixa,1,2,3,4,5
0,6,0.0,0.0,0.0,0.0,1.0
1,7,0.0,1.0,0.0,0.0,0.0
2,8,0.0,1.0,0.0,0.0,0.0
3,18,0.0,1.0,0.0,0.0,0.0
4,23,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...
60,111,1.0,0.0,0.0,1.0,0.0
61,112,0.0,0.0,0.0,1.0,0.0
62,113,0.0,1.0,1.0,1.0,0.0
63,114,0.0,1.0,1.0,1.0,1.0


In [ ]:
df_caixas_por_loja_highs_s5.columns = (['caixas'] + list(w.columns)) # os número das colunas são os nomes das lojas
df_caixas_por_loja_highs_s5

,caixas,13_EN_4,20_CX_4,31_CX_6,65_EN_4,9_CX_7
0,6,0.0,0.0,0.0,0.0,1.0
1,7,0.0,1.0,0.0,0.0,0.0
2,8,0.0,1.0,0.0,0.0,0.0
3,18,0.0,1.0,0.0,0.0,0.0
4,23,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...
60,111,1.0,0.0,0.0,1.0,0.0
61,112,0.0,0.0,0.0,1.0,0.0
62,113,0.0,1.0,1.0,1.0,0.0
63,114,0.0,1.0,1.0,1.0,1.0


#### Caixas

In [ ]:
print('Solver Highs')
total_caixas = 0
for col in w.columns:
    print(f'número de caixas para a chave_loja_buffer_onda "{col}": {df_caixas_por_loja_highs_s5[col].sum()}')
    total_caixas += df_caixas_por_loja_highs_s5[col].sum()
print(f'\ntotal de caixas: {total_caixas}')

Solver Highs
número de caixas para a chave_loja_buffer_onda "13_EN_4": 29.0
número de caixas para a chave_loja_buffer_onda "20_CX_4": 21.0
número de caixas para a chave_loja_buffer_onda "31_CX_6": 5.0
número de caixas para a chave_loja_buffer_onda "65_EN_4": 37.0
número de caixas para a chave_loja_buffer_onda "9_CX_7": 26.0

total de caixas: 118.0
